In [1]:
#coding: utf8
from __future__ import print_function, division
import sys
import numpy as np
from numpy.lib.recfunctions import stack_arrays
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.colors import ListedColormap
# Leave out the bright values, good for quiver overlays
inferno_c = ListedColormap(plt.cm.inferno.colors[:200], name="inferno_c")
from mpl_toolkits.axes_grid1.inset_locator import (zoomed_inset_axes,
                                                   inset_axes, mark_inset)
import healpy as hp
import scipy.optimize as sco
import scipy.interpolate as sci
import emcee
from tqdm import tqdm as tqdm

from anapymods3.healpy import (gaussian_on_a_sphere, get_binned_healpy_map, 
                               DecRaToThetaPhi, ThetaPhiToDecRa, wrap_angle,
                               rotator)
from anapymods3.plots import (ps_cmap, EquCoordsToMapCoords, cartzoom,
                              cartzoom_radius, make_astro_xaxis,
                              reconstruct_cartzoom_data)
from anapymods3.stats import prob2sigma

TODO:
1. Seed Funktion schreiben, auf's Beste hoffen...
2. Besten BG gefunden? Wie überprüfen??
3. Injiziertes Signal gefunden? s.o.?

# Code

## Event injection

In [2]:
def inject_bg(n, rnd):
    """
    Generate BG events' ra, dec coordinates uniformly on the sphere.
    
    Parameters
    ----------
    n : int
        Number of BG-like event positions to draw.
    rnd : np.random.RandomState instance
        Random number generator to use.
    
    Returns
    -------
    ev_ra, ev_dec : array-like, shape (n)
        BG-like event positions in radians.
    """
    ev_ra = rnd.uniform(0, 2. * np.pi, size=n)
    ev_dec = np.arccos(rnd.uniform(-1, 1, size=n)) - np.pi / 2.
    return ev_ra, ev_dec

def inject_signal(n, src_ra, src_dec, sigma, rnd):
    """
    Inject from a 2D uncorrelated gaussian with per event sigma
    
    Parameters
    ----------
    n : int
        Number of BG-like event positions to draw.
    src_ra, src_dec : float
        Single source position to inject signal at, in radian.
    sigma : float or array-like shape (n)
        Per event spatial uncertainty to introduce realistic variation.
        If float the same value is used for all events, if array it must have
        length ``n``.
    rnd : np.random.RandomState instance
        Random number generator to use.
        
    Returns
    -------
    ev_ras, ev_decs : array-like, shape (n)
        Signal-like event positions, drawn from the gaussian assumption.
    """
    if (src_ra < 0 or src_ra > 2. * np.pi or
            src_dec < -np.pi / 2. or src_dec > np.pi / 2.):
        raise ValueError("src_ra/src_dec outside range [0, 2pi]x[-pi/2,pi/2].")
    sigma = np.atleast_1d(sigma)
    if len(sigma) != 1 and len(sigma) != n:
        raise ValueError("sigma must be a single float or have length `n`.")
    # Lazy: Symmetric gaussian drawn at horizon rotated to target coordinates.
    # If len(sigma) == 1, then all events share the same uncertainty estimation.
    ev_ras, ev_decs = rnd.normal(0, sigma, size=(2, n))
    return rotator(ra1=np.zeros(n), dec1=np.zeros(n),
                   ra2=np.repeat(src_ra, n), dec2=np.repeat(src_dec, n),
                   ra3=ev_ras, dec3=ev_decs)

def get_evt_sigmas(n, rnd):
    """
    Generates sigma uncertainties per event from a tuned Rayleigh PDF
    
    Parameters
    ----------
    n : int
        Number of sigmas to draw.
    rnd : np.random.RandomState instance
        Random number generator to use.
        
    Returns
    -------
    sigmas : array-like, shape (n)
        Created sigma spatial event uncertainties in radian.
    """
    # Min at 0.3°, peakes at ~1° slight tail to 3°
    rvs = scs.rayleigh.rvs(loc=0.3, scale=0.7, size=n, random_state=rnd)
    return np.deg2rad(rvs) 

def make_sample(nbg, sig=None, rnd=None):
    """
    Create a test sample with BG and signal events.
    
    Parameters
    ---------
    nbg : int
    sig : dict, optional
        If not ``None``, must have keys:
        - "nsig", int or array-like:
          Number of signal events to generate. If single number and "ra", "dec"
          are multiple positions, then the same number is used for all sources.
        - "ra", "dec", float or array-like:
          Source position(s) in equatorial coordinates in radian.
        - "sigma", float or array-like:
          Used to randomize the source position using a symmetric 2D gaussian
          (as the used priors). The source position is drawn form the gaussian
          and then events are injected at this position. If single number and
          "ra", "dec" are multiple positions, then the same number is used for
          all sources. Assign ``0.`` if no src variation should be used.
          
        (Default: ``None``)
    
    rnd : np.random.RandomState, optional
        Random number generator instance. If ``None`` a new one with seed ``0``
        is created. (Default: ``None``)
        
    Returns
    -------
    ras, decs, sigmas : array-like
        Event positions and uncertainties in radian for both BG and signal.
    sig_idx : int array-like
        Indices indicating to which class the events belong. ``0`` are all BG
        events, then integer increments for each source position.
    """
    if rnd is None:
        rnd = np.random.RandomState()
    
    # Make background events
    ra_bg, dec_bg = inject_bg(nbg, rnd)
    sigma_bg = get_evt_sigmas(nbg, rnd)
    
    sig_idx = np.zeros(nbg, dtype=int)

    # Make signal events if desired
    ra_sig, dec_sig, sigma_sig = [], [], []
    if sig is not None:
        if not isinstance(sig, dict):
            raise ValueError("If sig is not None, must be a dict.")
        for key in ["nsig", "ra", "dec", "sigma"]:
            if key not in sig.keys():
                raise ValueError("sig is missing key '{}'".format(key))
        
        nsig = np.atleast_1d(sig["nsig"]).astype(int)
        src_ras = np.atleast_1d(sig["ra"])
        src_decs = np.atleast_1d(sig["dec"])
        src_sigmas = np.atleast_1d(sig["sigma"])
        nsrcs = len(src_ras)
        
        # If nsig or sigma is only a single number, use same val for all srcs
        if len(src_ras) != len(src_decs):
            raise ValueError("src ra and dec must have same length.")      
        if len(nsig) != nsrcs:
            if not len(nsig) == 1:
                raise ValueError("nsig must have same len as ra, dec or be a" +
                                 "single number.")
            nsig = np.repeat(nsig, repeats=nsrcs)
        if len(src_sigmas) != nsrcs:
            if not len(src_sigmas) == 1:
                raise ValueError("sigma must have same len as ra, dec or be a" +
                                 "single number.")
            src_sigmas = np.repeat(src_sigmas, repeats=nsrcs)
        
        # Inject events, draw src positions first to imitate src pos uncertainty
        src_ras = rnd.normal(loc=src_ras, scale=src_sigmas, size=nsrcs)
        src_decs = rnd.normal(loc=src_decs, scale=src_sigmas, size=nsrcs)
        print("sigma : [{}]. New true src locations are:".format(", ".join(
            ["{:.1f}".format(sigi) for sigi in np.rad2deg(src_sigmas)])))
        print("  RA  : [{}] in deg".format(", ".join(["{:.1f}".format(
            rai) for rai in np.rad2deg(src_ras)])))
        print("  DEC : [{}] in deg".format(", ".join(["{:.1f}".format(
            deci) for deci in np.rad2deg(src_decs)])))
        
        ra_sig, dec_sig, sigma_sig = [], [], []
        for i, (nsigi, srai, sdeci) in enumerate(zip(nsig, src_ras, src_decs)):
            sigma_sig.append(get_evt_sigmas(nsigi, rnd=rnd))
            ra_sigi, dec_sigi = inject_signal(nsigi, srai, sdeci,
                                              sigma=sigma_sig[-1], rnd=rnd)
            ra_sig.append(ra_sigi)
            dec_sig.append(dec_sigi)
            sig_idx = np.r_[sig_idx, np.ones(nsigi, dtype=int) * (i + 1)]

        ra_sig = np.concatenate(ra_sig)
        dec_sig = np.concatenate(dec_sig)
        sigma_sig = np.concatenate(sigma_sig)
    
    # Combine sample
    ev_ras = np.concatenate((ra_bg, ra_sig))
    ev_decs = np.concatenate((dec_bg, dec_sig))
    ev_sigmas = np.concatenate((sigma_bg, sigma_sig))
    
    assert len(sig_idx) == len(ev_ras) == len(ev_decs) == len(ev_sigmas)
    
    return ev_ras, ev_decs, ev_sigmas, sig_idx

## Scans & Plots

In [3]:
def sky_scan(NSIDE, data, lnpriormap=None, ra0=None, dec0=None,
             radius=np.deg2rad(5), ns_fix=False, box=None):
    """
    Profile scan the LLH for given fixed events ras, decs on a NSIDE map
    
    Parameters
    ----------
    NSIDE : int
        Resolution of the scan.
    data : array-like
        Holds fixed experimental data: [ev_ra, ev_dec, ev_sigma].
    lnpriormap : array-like or None
        If not None a prior is used for the ra, dec fit. Must be a list of maps
        [lnprior_map, ln_prior_grad_th_map, ln_prior_grad_phi_map] holding
        the healpy maps with the ln prior and the derivatives in theta and phi
        (already divided by sin(theta) per pixel). (default: None)
    ra0, dec0, radius : float, float, float
        If given, scan only the pixels in radius around (ra0, dec0).
    radius : float, optional
        Scan radius around the selected source. (Default: np.deg2rad(5))
    ns_fix : float or False
        If not False, do not profile scan ns, but keep it fixed at every pix.
    box : array-like, float or None
        See `llh_ratio` for info, gets simply passed to function.
        
    Returns
    -------
    ns_map, llh, grad_map
    """
    # If radius is given only scan so much pixels around ra0, dec0
    NPIX = hp.nside2npix(NSIDE)
    if ra0 is not None and dec0 is not None:
        if ra0 < 0 or ra0 > 2*np.pi or dec0 < -np.pi/2 or dec0 > np.pi/2:
            raise ValueError("ra0, dec0 must be in [0, 2pi]x[-pi/2, pi/2].")
        if radius <= 0:
            raise ValueError("radius must be > 0.")
        vec0 = hp.dir2vec(DecRaToThetaPhi(dec0, ra0))
        idx = hp.query_disc(NSIDE, vec0, radius, inclusive=True)
    else:
        idx = np.arange(NPIX)

    # Get pixel positions in ra, dec
    th_src, phi_src = hp.pix2ang(NSIDE, idx)
    dec_src, ra_src = ThetaPhiToDecRa(th_src, phi_src)

    ns_map = np.zeros(NPIX, dtype=float)
    llh_map = np.zeros(NPIX, dtype=float)
    grad_map = np.zeros((NPIX, 3), dtype=float)  # ns, ra, dec gradients
    
    if type(ns_fix) is bool:
        if ns_fix:
            raise ValueError("ns_fix can be False or a positive float.")
        # Profile scan by fitting ns in each pixel
        for i, idx_i in enumerate(idx):
            x0 = [1., ra_src[i], dec_src[i]]  # Simple 1D fit, seed > 0 is OK
            res = fit_llh_ratio(x0=x0, fix_ra_dec=True, data=data,
                                lnpriormap=lnpriormap, box=box)
            ns_map[idx_i] = res.x
            llh_map[idx_i] = -res.fun
            # Get all gradients, need to call llh_ratio again...
            x0[0] = res.x
            TS, grad = llh_ratio(params=x0, data=data,
                                 lnpriormap=lnpriormap, box=box)
            assert np.isclose(-res.fun, TS)
            grad_map[idx_i] = grad   # ns grad should always be <=0, test it!
    else:
        # Keep ns fixed for every pixel (all params fixed!)
        if ns_fix < 0.:
            raise ValueError("ns_fix must be >= 0 if not False.")
        for i, idx_i in enumerate(idx):
            TS, grad = llh_ratio(params=[ns_fix, ra_src[i], dec_src[i]],
                                 data=data, lnpriormap=lnpriormap, box=box)
            ns_map[idx_i] = ns_fix
            llh_map[idx_i] = TS
            grad_map[idx_i] = grad
        
    return ns_map, llh_map, grad_map


def sky_scan_full_radec(params, data, scan_idx, NSIDE, radius=np.deg2rad(5),
                        lnpriormap=None, box=None):
    """
    Profile scan using the full LLH around a single src ra, dec, keeping all
    other parameters fixed.
    
    Parameters
    ----------
    NSIDE : int
        Resolution of the scan.
    data : array-like
        Holds fixed experimental data: [ev_ra, ev_dec, ev_sigma].
    scan_idx : int
        Which of the sources to scan, is then using params [src_idx] to center
        the scan at.
    radius : float, optional
        Scan radius around the selected source. (Default: np.deg2rad(5))
    lnpriormap : array-like or None
        If not None a prior is used for the ra, dec fit. Must be a list of maps
        [lnprior_map, [ln_prior_grad_th_map], [ln_prior_grad_phi_map]] holding
        the healpy maps with the ln prior and the derivatives in theta and phi
        (already divided by sin(theta) per pixel). (default: None)
    box : array-like, float or None
        See `llh_ratio` for info, gets simply passed to function.
        
    Returns
    -------
    ns_map, llh, grad_map
    """
    ns_fix = True
    
    params, _, _ = _select_events(params, data, box=None)
    
    # If radius is given only scan so much pixels around ra0, dec0
    NPIX = hp.nside2npix(NSIDE)
    if type(scan_idx) is int:
        if scan_idx >= len(params[0]) or scan_idx < 0:
            raise ValueError("scan_idx must be a valid index to a param array.")
    else:
        raise ValueError("scan_idx must be an integer index.")

    ns0, ra0, dec0 = params[:, scan_idx]
    vec0 = hp.dir2vec(DecRaToThetaPhi(dec0, ra0))
    idx = hp.query_disc(NSIDE, vec0, radius, inclusive=True)

    # Get pixel positions in ra, dec
    th_src, phi_src = hp.pix2ang(NSIDE, idx)
    dec_src, ra_src = ThetaPhiToDecRa(th_src, phi_src)

    ns_map = np.zeros(NPIX, dtype=float)
    llh_map = np.zeros(NPIX, dtype=float)
    grad_map = np.zeros((NPIX, 3), dtype=float)  # ns, ra, dec gradients
    
    if type(ns_fix) is bool:
        if ns_fix:
            raise ValueError("ns_fix can be False or a positive float.")
        # Profile scan by fitting ns in each pixel
        for i, idx_i in enumerate(idx):
            x0 = [1., ra_src[i], dec_src[i]]  # Simple 1D fit, seed > 0 is OK
            res = fit_llh_ratio(x0=x0, fix_ra_dec=True, data=data,
                                lnpriormap=lnpriormap, box=box)
            ns_map[idx_i] = res.x
            llh_map[idx_i] = -res.fun
            # Get all gradients, need to call llh_ratio again...
            x0[0] = res.x
            TS, grad = llh_ratio(params=x0, data=data,
                                 lnpriormap=lnpriormap, box=box)
            assert np.isclose(-res.fun, TS)
            grad_map[idx_i] = grad   # ns grad should always be <=0, test it!
    else:
        # Keep ns fixed for every pixel (all params fixed!)
        if ns_fix < 0.:
            raise ValueError("ns_fix must be >= 0 if not False.")
        for i, idx_i in enumerate(idx):
            TS, grad = llh_ratio(params=[ns_fix, ra_src[i], dec_src[i]],
                                 data=data, lnpriormap=lnpriormap, box=box)
            ns_map[idx_i] = ns_fix
            llh_map[idx_i] = TS
            grad_map[idx_i] = grad
        
    return ns_map, llh_map, grad_map

## LLH

In [4]:
def pdf_sig(src_ras, src_decs, ras, decs, sigmas):
    """
    Gaussian 2D PDF for signal (as injected)
    
    Returns
    -------
    pdf : array-like, shape (nsrcs, nevts)
        Signal PDF values per source for all events.
    grad : array-like, shape (2 * nsrcs, nevts)
        Gradients [dra, ddec] for each src_ras, src_decs for each ras, decs.
    """
    # Broadcast for multiple srcs: (nsrcs, nevts)
    src_ras = np.atleast_1d(src_ras)[:, None]
    src_decs = np.atleast_1d(src_decs)[:, None]
    ras = np.atleast_1d(ras)[None, :]
    decs = np.atleast_1d(decs)[None, :]
    sigmas = np.atleast_1d(sigmas)[None, :]
    
    assert src_ras.shape == src_decs.shape
    assert ras.shape == decs.shape == sigmas.shape
    if len(src_ras.shape) != 2 or len(ras.shape) != 2:
        raise ValueError("src and ev positions were no 1D arrays or floats.")
    
    cos_dist = (np.cos(src_ras - ras) * np.cos(src_decs) * np.cos(decs) +
                np.sin(src_decs) * np.sin(decs))
    cos_dist = np.atleast_1d(np.clip(cos_dist, -1, 1))
    # Angular distance squared for 2D gaussian
    dist = np.arccos(cos_dist)
    sig2 = sigmas**2
    pdf = 0.5 / np.pi / sig2 * np.exp(-0.5 * dist**2 / sig2)
    
    # Use chain rule to get gradient in src_ras, src_decs
    m = (cos_dist > -1.) & (cos_dist < 1.)
    _grad = pdf * dist / sig2
    # At -1, 1 the naive calculation diverges, but the gradient exists, so mask it
    _grad[m] = _grad[m] / np.sqrt(1. - cos_dist[m]**2)
    grad_ra = _grad * (np.sin(ras - src_ras) * np.cos(src_decs) * np.cos(decs))
    grad_dec = _grad * (np.cos(src_ras - ras) * -np.sin(src_decs) *
                np.cos(decs) + np.cos(src_decs) * np.sin(decs))
    return pdf, np.vstack([grad_ra, grad_dec])


def pdf_bg(src_decs, decs):
    """
    Uniform PDF for BG (as injected)
    
    Returns
    -------
    pdf : array-like, shape (nsrcs, nevts)
        BG PDF values per source for all events.
    grad : array-like, shape (2 * nsrcs, nevts)
        Gradients [dra, ddec] for each src_ra, src_dec for each ras, decs.
    """
    src_decs = np.atleast_1d(src_decs)[:, None]
    decs = np.atleast_1d(decs)[None, :]
   
    pdf = np.ones((src_decs.shape[0],
                   decs.shape[1]), dtype=float) / (4. * np.pi)
    grad = np.vstack([np.zeros_like(pdf), np.zeros_like(pdf)])
    
    return pdf, grad


def pdf_sob(src_ras, src_decs, ras, decs, sigmas):
    """
    Ratio of signal and bg PDFs. Gradient calculated with ratio rule.
    
    Returns
    -------
    sob : array-like, shape (nsrcs, nevts)
        Signal over background PDF ratios per event and per source.
    grad : array-like, shape (2 * nsrcs, nevts)
        Gradients [dra, ddec] for each src_ras, src_decs for each ras, decs.
    """
    sig_pdf, sig_grad = pdf_sig(src_ras, src_decs, ras, decs, sigmas)
    bg_pdf, bg_grad = pdf_bg(src_decs, decs)

    assert sig_pdf.shape == bg_pdf.shape
    assert sig_grad.shape == bg_grad.shape

    # Repeat PDF values, because gradient is both for ra and dec
    sig_pdf_ = np.vstack((sig_pdf, sig_pdf))
    bg_pdf_ = np.vstack((bg_pdf, bg_pdf))
    # Quotient rule (u / v)' = (u'v - uv') / v**2, no special cases for ra/dec
    # Here same as: sig_grad / bg_pdf, because bg_pdf is const -> bg_grad = 0
    grad = (sig_grad * bg_pdf_ - sig_pdf_ * bg_grad) / bg_pdf_**2

    assert np.allclose(grad, sig_grad / bg_pdf_)
    
    return sig_pdf / bg_pdf, grad


def _select_events(params, data, box):
    """
    Select all events or in boxes around each source.
    
    Parameters
    ----------
    See ``llh_ratio`` function docs for parameters.
    
    Returns
    -------
    params : tuple, shape (3, nsrcs)
        Parameter values ``([ns0, ...], [src_ra0, ...], [src_dec0, ...])``.
    data : tuple, shape (3, nevts)
        Holds fixed experimental data: ``([ev_ras], [ev_decs], [ev_sigmas])``.
    (Nall, Nsel, nsrcs) : tuple
        Number of all original events, number of selected events and number of
        sources, inferred from input params.
    """
    if len(params) != 3:
        raise ValueError("params must be (ns, src_ras, src_decs)")
    if len(data) != 3:
        raise ValueError("data must be  (ras, decs, sigmas)")
    ns, src_ras, src_decs = [np.atleast_1d(pi) for pi in params]
    ras, decs, sigmas = [np.atleast_1d(di) for di in data]
    if np.any(np.abs(src_decs) > np.pi / 2.):
        raise ValueError("src_dec(s) found outside valid range [-pi/2, pi/2].")
    nsrcs = len(src_ras)
    Nall = float(len(ras))
    
    assert len(ras) == len(decs) == len(sigmas)
    assert len(ns) == len(src_ras) == len(src_decs)
    
    # We need to wrap ra back to it's main range, otherwise box mode will break
    # This might not work if we don't use analytic gradients, because the fitter
    # only would see the euclidean diff in ra, which suddenly is 2pi.
    src_ras = np.mod(src_ras - 2. * np.pi, 2. * np.pi)
    
    if box is not None:
        box = 0.5 * np.atleast_1d(box)  # Centered box interval.
        if len(box) != nsrcs:
            if len(box) != 1:
                raise ValueError("If len(box)!=nsrcs it must be a single nr.")
            box = np.repeat(box, repeats=nsrcs)
      
        # Closer to the poles we widen the box to get all significant SoBs
        max_height = np.abs(src_decs) + box
        # Avoid cases of cos(dec)<=0
        band = np.ones_like(box) * np.pi
        m = (max_height < np.deg2rad(89))
        band[m] = box[m] / np.cos(max_height[m])

        # Broadcast version, faster if enough memory
        box = box[:, None]
        band = band[:, None]
        # Center around src and test symmetric box
        ras_ = (ras[None, :] - src_ras[:, None])
        ra_m = (((ras_ > -band) & (ras_ < band)) | 
                ((ras_ > 2. * np.pi - band) | (ras_ < -2. * np.pi + band)))
        dec_m = ((decs[None, :] > src_decs[:, None] - box) &
                 (decs[None, :] < src_decs[:, None] + box))
        
        band_m = np.any(ra_m & dec_m, axis=0)

        Nsel = float(np.sum(band_m))
        ras = ras[band_m]
        decs = decs[band_m]
        sigmas = sigmas[band_m]
    else:
        Nsel = Nall
        
    return [ns, src_ras, src_decs], [ras, decs, sigmas], (Nall, Nsel, nsrcs)


def llh_ratio(params, data, lnpriormap=None, box=None):
    """
    Standard PS likelihood ratio, depending on ns and {src_ras, src_decs}.
    TS = 2 * [ln(L1) - ln(L0)]
    
    Parameters
    ----------
    params : tuple, shape (3, nsrcs)
        Parameter values ``([ns0, ...], [src_ra0, ...], [src_dec0, ...])``.
    data : tuple, shape (3, nevts)
        Holds fixed experimental data: ``([ev_ras], [ev_decs], [ev_sigmas])``.
    lnpriormap : tuple, shape (3, ...) or None
        If not None a prior is used for the ra, dec fit. Must be a list of maps
        ``(lnprior_map, [ln_prior_grad_th_maps], [ln_prior_grad_phi_map])``
        holding the single summed ln(prior) healpy map and and the nsrcs
        derivate maps in in dtheta and dphi (already divided by sin(theta) per
        pixel). (default: None)
    band : array-like or None
        If not ``None`` give the width of a dec band centered at each src in
        which events are selected to speed up PDF evaluation. (Default: None)
    
    Returns
    -------
    TS : float
    grad : array-like
    """      
    # Get selected and prepared events and calc SoBs and gradients
    params, data, (Nall, Nsel, nsrcs) = _select_events(params, data, box)
    (ns, src_ras, src_decs), (ras, decs, sigmas) = params, data    
    sob, sob_grad = pdf_sob(src_ras, src_decs, ras, decs, sigmas)
    
    # Calculate TS = 2 * (ln(L1) - ln(L0)), broadcast for multiple ns (per src)
    ns = ns[:, None]
    ns_sum = np.sum(ns)
    a = np.sum(ns * sob, axis=0) - ns_sum   # shape (nevts)
    assert len(a) == len(ras) == Nsel

    TS = np.sum(np.log1p(a / Nall))         # float
      
    # Get gradients in all params (nsrcs * {ns, ra, dec}) using chain rule
    grad_denom = 1. / (a + Nall)                              # shape (nsrcs)
    grad_ns = np.sum((sob - 1.) * grad_denom, axis=1)         # shape (nsrcs)
    # sob_grad is [grad_ra, grad_dec] so repeat ns for both.  # shape (nsrcs)
    grad_radec = np.sum(np.vstack((ns, ns)) * sob_grad * grad_denom, axis=1)
    
    # If we use box mode, TS & ns gradients need to account for all S=0 events
    if Nsel < Nall:
        diffN = Nall - Nsel
        assert diffN > 0
        TS += diffN * np.log1p(-ns_sum / Nall)  # float
        grad_ns -= diffN / (Nall - ns_sum)      # shape (nsrcs)
        assert len(grad_ns) == nsrcs
    
    # As we operate on spherical coordinates, we need to scale the ra gradients
    cos_decs = np.cos(src_decs)
    m = ~np.isclose(cos_decs, 0.)
    # At the poles the gradient is ill defined anyway
    grad_ra = np.zeros(nsrcs, dtype=float)
    grad_ra[m] = grad_radec[:nsrcs][m] / cos_decs[m]
    grad_radec = np.concatenate((grad_ra, grad_radec[nsrcs:]))
    
    assert len(grad_ns) == nsrcs
    assert len(grad_ra) == nsrcs
    assert len(grad_radec) == 2 * nsrcs
        
    # Include values from prior maps if we have any
    if lnpriormap is not None:
        if len(lnpriormap) != 3:
            raise ValueError("Need concatenated map arrays (sum(ln(prior)), " +
                             "[d/dth ln(priors)], [d/phi ln(priors)]).")
        
        lnprior, lnpriors_dth, lnpriors_dphi = lnpriormap
        if (hp.maptype(lnpriors_dth) != nsrcs or
                hp.maptype(lnpriors_dth) != nsrcs):
            raise ValueError("dth, dphi priors need one gradient map per src.")
        
        # Get prior, all srcs are summed up, single map is sufficient
        src_ths, src_phis = DecRaToThetaPhi(src_decs, src_ras)
        TS += np.sum(hp.get_interp_val(m=lnprior, theta=src_ths, phi=src_phis))
        
        lnprior_grad_ra = np.zeros(nsrcs, dtype=float)
        lnprior_grad_dec = np.zeros(nsrcs, dtype=float)
        # Get gradients from dth, dphi maps per source
        for i, (th, phi) in enumerate(zip(src_ths, src_phis)):
            # No sign for ra: ra = phi
            lnprior_grad_ra[i] = hp.get_interp_val(m=lnpriors_dphi[i],
                                                   theta=th, phi=phi)
            # Negative sign for dec: th = np.pi / 2 - dec -> dth/ddec = -1
            lnprior_grad_dec[i] = -hp.get_interp_val(m=lnpriors_dth[i],
                                                     theta=th, phi=phi)

        # Add back to LLH gradients
        grad_radec += np.concatenate((lnprior_grad_ra, lnprior_grad_dec))
            
    return 2. * TS, 2. * np.concatenate([grad_ns, grad_radec])


def fit_llh_ratio_radec(x0, data, fix_ra_dec=False, lnpriormap=None, box=None):
    """
    Fit the multi source likelihood ratio defined in `llh_ratio`.
    Special wrapper case, where only all parameters, or {ns} but not the
    {ra, decs} can be fitted.
    For more control on which parameter gets fitted, use `fit_llh_ratio`.
    
    Parameters
    ----------
    x0 : array-like
        Start values [ns0, ra0, dec0] for the fit. If ``fix_ra_dec`` is
        ``True`` only the {ns} values are fitted.
    data : array-like
        Holds fixed experimental data: [ev_ra, ev_dec, ev_sigma].
    fix_ra_dec : bool
        If True x[1], x[2] = ra_fix, dec_fix are held fixed and only ns is
        minimizedto. Else ns, ra, dec are fitted and x0 os the seed for all
        parameters. (default: False)
    lnpriormap : array-like or None
        If not None a prior is used for the ra, dec fit. Must be a list of maps
        [lnprior_map, ln_prior_grad_th_map, ln_prior_grad_phi_map] holding
        the healpy maps with the ln prior and the derivatives in theta and phi
        (already divided by sin(theta) per pixel). (default: None)
    box : array-like or None
        If not ``None`` give the width of a dec box centered at each src in
        which events are selected to speed up PDF evaluation. (Default: None)
    """
    # If x0 is badly formatted it crashes in `fit_llh_ratio`, so no check here
    nsrcs = len(np.atleast_1d(x0[0]))
    fitted = np.ones((3, nsrcs), dtype=bool)
    if fix_ra_dec:
        fitted[1:] = False

    return fit_llh_ratio(x0, data, fitted, lnpriormap=lnpriormap, box=box)


def fit_llh_ratio_onesrc(x0, data, src_idx, fix_ra_dec=False, lnpriormap=None,
                         box=None):
    """
    Fit the multi source likelihood ratio defined in `llh_ratio`.
    Special wrapper case, where only a single source is fitted in (ns, ra, dec),
    but other src parameters can be held fixed.
    If fix_radec is True, then only ns is fitted for that single src (1D fit).
    For more control on which parameter gets fitted, use `fit_llh_ratio`.
    
    Parameters
    ----------
    x0 : array-like
        Start values [ns0, ra0, dec0] for the fit. If ``fix_ra_dec`` is
        ``True`` only the {ns} values are fitted.
    data : array-like
        Holds fixed experimental data: [ev_ra, ev_dec, ev_sigma].
    src_idx : int
        Select the source to fit ns or (ns, ra, dec) for with every other
        parameter held fixed.
    fix_ra_dec : bool
        If True only ns is fitted, else (ns, ra, dec). (default: False)
    lnpriormap : array-like or None
        If not None a prior is used for the ra, dec fit. Must be a list of maps
        [lnprior_map, ln_prior_grad_th_map, ln_prior_grad_phi_map] holding
        the healpy maps with the ln prior and the derivatives in theta and phi
        (already divided by sin(theta) per pixel). (default: None)
    box : array-like or None
        If not ``None`` give the width of a dec box centered at each src in
        which events are selected to speed up PDF evaluation. (Default: None)
    """
    # If x0 is badly formatted it crashes in `fit_llh_ratio`, so no check here
    nsrcs = len(np.atleast_1d(x0[0]))
    fitted = np.zeros((3, nsrcs), dtype=bool)
    if fix_ra_dec:
        fitted[0, src_idx] = True
    else:
        fitted[:, src_idx] = True

    return fit_llh_ratio(x0, data, fitted, lnpriormap=lnpriormap, box=box)


def fit_llh_ratio(x0, data, fitted, lnpriormap=None, box=None):
    """
    Only fit in the parameters not declared fixed, but still using the complete
    multi source likelihood.
    
    Parameters
    ----------
    x0 : array-like, shape (3, nsrcs)
        Start values [ns0, ra0, dec0] for the fit. Bool mask ``fixed``
        determines which parameter is actually fitted.
    data : array-like, shape (3, nevts)
        Holds fixed experimental data: [ev_ra, ev_dec, ev_sigma].
    fitted : array-like, bool, shape (3, nsrcs)
        Indices indicating which parameters sould be fitted, others are fixed.
    lnpriormap : array-like or None
        If not None a prior is used for the ra, dec fit. Must be a list of maps
        [lnprior_map, ln_prior_grad_th_map, ln_prior_grad_phi_map] holding
        the healpy maps with the ln prior and the derivatives in theta and phi
        (already divided by sin(theta) per pixel). (default: None)
    box : array-like or None
        If not ``None`` give the width of a dec box centered at each src in
        which events are selected to speed up PDF evaluation. (Default: None)
    """
    assert len(data[0]) == len(data[1]) == len(data[2])

    if len(x0) != 3:
        raise ValueError("x0 must be [ns0, ra0, dec0].")
    if len(fitted) != 3:
        raise ValueError("fitted must be [ns_mask, ra_mask, dec_mask].")
        
    x0 = [np.atleast_1d(x0i) for x0i in x0]
    fitted = [np.atleast_1d(fi) for fi in fitted]
    nsrcs = len(x0[0])
    if not np.all([len(x0i) == nsrcs for x0i in x0[1:]]):
        raise ValueError("{ns, ra, dec} arrays in x0 must be the same size.")
    if not np.all([len(fi) == nsrcs for fi in fitted]):
        raise ValueError("All fitted masks must be the same size.")
    
    x0 = np.array(x0)
    fitted = np.array(fitted)
    if not fitted.dtype == bool:
        raise ValueError("fitted must be a list of bool arrays.")
    
    # Just evaluate the LLH if all params are fixed
    if not np.any(fitted):
        raise ValueError("All parameters fixed, just evaluating the LLH:")
        return llh_ratio(x0, data, lnpriormap, box)
        
    # Stitch together boundary blocks for ns, ra, dec
    bnds = [[0, len(data[0])],          # ns
            [-2. * np.pi, 4. * np.pi],  # ra, pad interval for correct wrapping
            [-np.pi / 2., np.pi / 2.]]  # dec

    assert len(bnds) == len(np.sum(fitted, axis=1))
    bounds = np.concatenate([ni * bnd for ni, bnd in
        zip(np.sum(fitted, axis=1), bnds)]).reshape(np.sum(fitted), 2)
    
    # Some fixed stuff reused in fit
    x0_fit = x0[fitted]
    ns_fit, ra_fit, dec_fit = np.sum(fitted, axis=1)
    fitted_flat = np.ravel(fitted)
        
    def fitfunc(params):
        """ Wrapper to minimize the negative LLH """
        # Rebuild whole parameter set
        x0[fitted] = params
        TS, grad = llh_ratio(x0, data, lnpriormap, box)
        
        # Select gradients
        return -TS, -grad[fitted_flat]
        
    # Fit until we get a succesful result, but stop at maxiter
    maxiter = 10
    tol = 1e-14
    fit_opts = {"ftol": tol, "gtol": 1e-8}
    for _ in range(maxiter):
        # Start with a single L-BFGS-B fit
        res = sco.minimize(fitfunc, x0=x0_fit, jac=True, bounds=bounds,
                           options=fit_opts, tol=tol, method="L-BFGS-B")
        if res.success:
            break
        
        # If not succesful, try different fitter on same seed first
        print("  ! Bad fit, retry with other fitter.")
        res = sco.minimize(fitfunc, x0=res.x, jac=True, bounds=bounds,
                           method="SLSQP", tol=tol, options={"ftol": tol})
        if res.success:
            break
        
        # If still not succesful, change seeds a bit towards the gradients
        print("  res.message", res.message)
        print("  ! Bad fit, retry with other seed.")
        ns0 = res.x[:ns_fit]
        ra0 = res.x[ns_fit:ns_fit+ra_fit]
        dec0 = res.x[-dec_fit:]
        dns0 = res.jac[:ns_fit]
        dra0 = res.jac[ns_fit:ns_fit+ra_fit]
        ddec0 = res.jac[-dec_fit:]
        
        # Don't seed {ns} very close to 0, we might get stuck there because
        # {ra, dec} gradients get zero everywhere
        # ns0 = np.clip(np.random.uniform(ns0 - 0.1, ns0 + 0.1, size=ns_fit),
        #               0.1, None)
        
        # Just wiggle a bit at the direction in sub deg range
        # dra0 = np.random.uniform(*np.deg2rad([-0.1, 0.1]), size=ra_fit)
        # ddec0 = np.random.uniform(*np.deg2rad([-0.1, 0.1]), size=dec_fit)
        
        # Go manually a little bit along the current gradients
        ns0 = np.clip(ns0 + np.clip(dns0, -0.1, 0.1),
                      bounds[:ns_fit, 0], bounds[:ns_fit, 1])
        ra0 = np.clip(ra0 + np.clip(dra0,*np.deg2rad([-0.1, 0.1])),
                      bounds[ns_fit:ns_fit+ra_fit, 0],
                      bounds[ns_fit:ns_fit+ra_fit, 1])
        dec0 = np.clip(dec0 + np.clip(ddec0,*np.deg2rad([-0.1, 0.1])),
                       bounds[-dec_fit, 0], bounds[-dec_fit:, 1])

        # Combine to new seed vector
        x0_fit = np.concatenate((ns0, ra0, dec0))
        assert len(x0_fit) == ns_fit + ra_fit + dec_fit == np.sum(fitted_flat)
        
    return res


def map_der(m):
    """
    Calculate a map derivative in th and phi directly: Get the derivative on a
    regular grid, then turn them into a map.
    We need to do this numerically using map interpolations, but we avoid
    ringing effects due to limited resolution in alm expansion.
    
    Parameters
    ----------
    m : array-like
        Valid healpy map.
        
    Returns
    -------
    grad_phi : array-like, shape (len(m))
        Gradient map in phi, already divided by sin(theta) per pixel.
    grad_th : array-like, shape (len(m))
        Gradient map in theta.
    """
    NSIDE = hp.get_nside(m)

    # Set stepsize (arbitrary) to the next better resolution (angle in radian)
    h = hp.nside2resol(2 * NSIDE)
    
    # Create new (phi, th) gridpoints
    phis = np.linspace(0, 2 * np.pi, int(np.ceil(2 * np.pi / h) + 1))
    ths = np.linspace(0, np.pi, int(np.ceil(np.pi / h) + 1))
    sin_ths = np.sin(ths)[:, None]
    
    # Wrap to get the symmetric gradients for all inner points
    phis = np.r_[phis[-1], phis, phis[0]]  # Periodic in phi
    ths = np.r_[ths[1], ths, ths[-2]]      # Going around the poles in theta
    
    # Get map values at new (phi, th) grid
    phis_, ths_ = np.meshgrid(phis, ths)
    PHIS, THS = map(np.ravel, [phis_, ths_])
    vals = hp.get_interp_val(m=m, phi=PHIS, theta=THS).reshape(phis_.shape)
    
    # Get the gradients using symmetric finite elemts
    grad_phi = (vals[:, 2:] - vals[:, :-2])[1:-1] / (2. * h)
    grad_th = (vals[2:] - vals[:-2])[:, 1:-1] / (2. * h)
    
    # Gradient in phi must be divided by sin(th) for the spherical gradient
    mask = (sin_ths == 0.)
    sin_ths[mask] = 1.  # At the poles the phi gradient is useless anyway
    grad_phi = grad_phi / sin_ths
    
    # Recombine to a map by averaging all gradients falling in a pixel
    PHIS, THS = map(np.ravel, np.meshgrid(phis[1:-1], ths[1:-1]))
    idx = hp.ang2pix(NSIDE, THS, PHIS)
    grad_phi = np.ravel(grad_phi)
    grad_th = np.ravel(grad_th)
    
    # Get count map for averaging
    NPIX = hp.nside2npix(NSIDE)
    count_map = np.bincount(idx, minlength=NPIX)
    count_map[count_map == 0] = 1.

    # Average gradients
    grad_phi = np.bincount(idx, minlength=NPIX, weights=grad_phi) / count_map
    grad_th = np.bincount(idx, minlength=NPIX, weights=grad_th) / count_map
        
    return grad_phi, grad_th


def get_seed_scan(src_ras, src_decs, radii, NSIDE, data, lnpriormap=None,
                  box=None):
    """
    Do a low res skyscan in a region around each source and profile fit the ns
    parameter for this source only for each pixel.
    The pixel with the best TS is used for the global fit over all {ns, ra, dec}
    parameters in the stacked LLH.
    
    This is an approximation only, because due to stacking the source regions
    are correlated and the single ns fit might not represent the location and
    nsj for the stacking best fit.
    
    Parameters
    ----------
    src_ras, src_decs : array-like, shape (nsrcs)
        Src positions in equtorial coordinates in radian.
    radii : array-like
        Radii in radian around the centers of each source for which pixel with
        the given ``NSIDE`` resolution are built and walkers are placed at.
    NSIDE : int
        Healpy resolution parameter. Together with radii, this determines the
        number of pixels scanned per source region.
    data : tuple, shape (3, nevts)
        Holds fixed experimental data: ``([ev_ras], [ev_decs], [ev_sigmas])``.
    lnpriormap : tuple, shape (3, ...) or None
        If not None a prior is used for the ra, dec fit. Must be a list of maps
        [lnprior_map, ln_prior_grad_th_map, ln_prior_grad_phi_map] holding
        the healpy maps with the ln prior and the derivatives in theta and phi
        (already divided by sin(theta) per pixel). (default: None)
    box : array-like or None
        If not ``None`` give the width of a dec box centered at each src in
        which events are selected to speed up PDF evaluation. (Default: None)
        
    Returns
    -------
    ns_seed, ra_seed, dec_seed : array-like, shape (nsrcs)
        Seed positions {ns, ra, dec} for each source to be used in the fitter
        which will most likely yield the global minimum.
    """
    # Check params and data format sanity
    (_, src_ras, src_decs), data, (_, _, nsrcs) = _select_events(
        (len(src_ras) * [-1], src_ras, src_decs), data, box=None)
    radii = np.atleast_1d(radii)
    if len(radii) != nsrcs:
        raise ValueError("radii must have length nsrcs.")
        
    if lnpriormap is not None:
        if len(lnpriormap) != 3:
            raise ValueError("Need concatenated map arrays (sum(ln(prior)), " +
                             "[d/dth ln(priors)], [d/phi ln(priors)]).")
        
        lnprior, lnpriors_dth, lnpriors_dphi = lnpriormap
        if (hp.maptype(lnpriors_dth) != nsrcs or
                hp.maptype(lnpriors_dphi) != nsrcs):
            raise ValueError("dth, dphi priors need one gradient map per src.")
    
    if box is None:
        box = nsrcs * [None]
    else:
        box = np.atleast_1d(box)
        if len(box) == 1:
            box = np.repeat(box, repeats=nsrcs)
        elif len(box) != nsrcs:
            raise ValueError("box can be None, single float or length nsrcs.")
    
    vecs = hp.ang2vec(*DecRaToThetaPhi(src_decs, src_ras))  # shape (nsrcs, 3)
    ns_seed, phi_seed, th_seed = np.zeros((3, nsrcs), dtype=float)
    ns0 = 2.  # Standard 1D fits per pixel, simple seed >> 0 is OK
    fitted = [[True], [False], [False]]
    
    assert len(ns_seed) == len(phi_seed) == len(th_seed) == len(vecs) == nsrcs
    
    for j, (veci, ri) in enumerate(zip(vecs, radii)):
        # Get selected scan pixel positions in ra, dec
        idx = hp.query_disc(NSIDE, veci, ri, inclusive=True)
        th_scan, phi_scan = hp.pix2ang(NSIDE, idx)
        dec_scan, ra_scan = ThetaPhiToDecRa(th_scan, phi_scan)
        
        # Setup priormaps for the correct src. Use the summed prior map, to get
        # some of the possible src correlation at least.
        lnpriormap_ = (lnpriormap[0], [lnpriormap[1][j]], [lnpriormap[2][j]])

        # Scan each pixel -> Try to run in multiprocessing?
        TS, ns = np.zeros((2, len(idx)), dtype=float)
        for i, idx_i in enumerate(idx):
            res = fit_llh_ratio(x0=[ns0, ra_scan[i], dec_scan[i]],
                                data=data, lnpriormap=lnpriormap_,
                                box=box[j], fitted=fitted)
            TS[i], ns[i] = res.fun, res.x[0]
            
        # Get best pixel position. Minimum because we fitted negLLH
        best_idx = np.argmin(TS)
        ns_seed[j] = ns[best_idx]
        th_seed[j], phi_seed[j] = hp.pix2ang(NSIDE, idx[best_idx])

    # Return best seed {ra, dec} per source
    dec_seed, ra_seed = ThetaPhiToDecRa(th_seed, phi_seed)
    return ns_seed, ra_seed, dec_seed

# Testplots

## Test multisource fit with seeding

In [ ]:
rnd_gen = np.random.RandomState(43374563)
nbg = int(1e5)
# src_ras = np.deg2rad([40, 20, 277, 300, 310])  # "pass2" setting
src_ras = np.deg2rad([40, 20, 359, 0.1, 1])
src_decs = np.deg2rad([70, 6, -80, -35, 0])
# nsig = np.arange(len(src_ras)) + 1
nsig = np.zeros_like(src_ras, dtype=int)
sig = {"nsig": nsig, "ra": src_ras, "dec": src_decs, "sigma": 0.}
ras, decs, sigmas, idx = make_sample(nbg, sig=sig, rnd=rnd_gen)

data = (ras, decs, sigmas)
nsrcs = len(src_ras)

In [ ]:
%%time
# Construct priors for visual comparison only (see note on top)
NSIDE_prior = 512
src_ths, src_phis = DecRaToThetaPhi(src_decs, src_ras)
sigma0 = np.deg2rad([1., 1.2, 0.7, 1., 1.5])
    
ln_priors = []
ln_priors_dphi = []
ln_priors_dth = []
for i, (th0i, phi0i, sigma0i) in enumerate(zip(src_ths, src_phis, sigma0)):
    print(u"Making prior at: RA={:.2f}°, DEC={:.2f}°, SIGMA={:.3f}°".format(
        np.rad2deg(ThetaPhiToDecRa(th0i, phi0i)[1])[0], 
        np.rad2deg(ThetaPhiToDecRa(th0i, phi0i)[0])[0], np.rad2deg(sigma0i)))
    prior = gaussian_on_a_sphere(mean_th=th0i, mean_phi=phi0i, 
                                 clip=5., NSIDE=NSIDE_prior, sigma=sigma0i)[0]
    # Avoid taking the log of 0, use the smallest valid map number instead
    prior[prior <= 0] = np.amin(prior[prior > 0])
    ln_prior = np.log(prior)

    # Make src_ras, src_decs more realistic by setting it to the map maximum
    idx = np.argmax(ln_prior)
    src_ths[i], src_phis[i] = hp.pix2ang(NSIDE_prior, idx)
    src_decs[i], src_ras[i] = ThetaPhiToDecRa(src_ths[i], src_phis[i])
    print("  New src pos at maximum of the prior map: " +
          "RA={:.2f}°, DEC={:.2f}°".format(*np.rad2deg((src_ras[i], src_decs[i]))))

    # Get the log derivatives from the finite elements method
    ln_prior_dphi, ln_prior_dth = map_der(ln_prior)
    
    ln_priors.append(ln_prior)
    ln_priors_dphi.append(ln_prior_dphi)
    ln_priors_dth.append(ln_prior_dth)
    
lnpriormap = (np.sum(ln_priors, axis=0), ln_priors_dth, ln_priors_dphi)

Get seeds from local scans. First with low res seed scan

In [ ]:
%%time
NSIDE_seed = 32
radii = 3. * sigma0
box = np.deg2rad(15)

seeds = get_seed_scan(src_ras, src_decs, radii, NSIDE_seed, data,
                      lnpriormap=lnpriormap, box=box)
ns_seed, ra_seed, dec_seed = seeds
[print(si) for si in seeds]

In [ ]:
i = 0
lnpriormap_ = (lnpriormap[0], [lnpriormap[1][i]], [lnpriormap[2][i]])
res = fit_llh_ratio_radec(x0=[ns_seed[i], ra_seed[i], dec_seed[i]], data=data,
                          fix_ra_dec=False, lnpriormap=lnpriormap_, box=box)
res

Refit with higher res scan, we should be close to the other fit everything works OK.

In [ ]:
%%time
NSIDE_seed = 128
radii = 3. * sigma0
box = np.deg2rad(15)

seeds_hr = get_seed_scan(src_ras, src_decs, radii, NSIDE_seed, data,
                         lnpriormap=lnpriormap, box=box)
ns_seed_hr, ra_seed_hr, dec_seed_hr = seeds_hr
[print(si) for si in seeds_hr]

In [ ]:
i = 0
lnpriormap_ = (lnpriormap[0], [lnpriormap[1][i]], [lnpriormap[2][i]])
res = fit_llh_ratio_radec(x0=[ns_seed_hr[i], ra_seed_hr[i], dec_seed_hr[i]], data=data,
                          fix_ra_dec=False, lnpriormap=lnpriormap_, box=box)
res

Scan for all parameters

In [ ]:
fitted = np.ones((3, len(ns_seed)), dtype=bool)
res = fit_llh_ratio(x0=[ns_seed, ra_seed, dec_seed], data=data,
                    fitted=fitted, lnpriormap=lnpriormap, box=box)
print(res.x[:nsrcs])
print(res.x[nsrcs:2*nsrcs])
print(res.x[-nsrcs:])
print(res)

Profile scan {nsj, raj, decj} around each source j's region keeping all other parameters {ns, ra, dec} at the best fit positions from the multifit.
The scan should end up in the same minimum for {nsj, raj, decj} as the multi fit.

In [ ]:
# %%time
NSIDE_plot = 128

for src_idx in tqdm(range(len(src_decs))[:]):
    ra0_ = src_ras[src_idx]
    dec0_ = src_decs[src_idx]
    lnpriormap_ = (lnpriormap[0], [lnpriormap[1][src_idx]],
                   [lnpriormap[2][src_idx]])
    zoom = radii[src_idx]

    # Scan complete LLH in local {ra, dec}, holding all {ns} and the other
    # {ra, dec} fixed (profile scan around the minimum in {ra, dec})
    nsmap, llhmap, gradmaps = sky_scan(NSIDE=NSIDE_plot, data=data,
         lnpriormap=lnpriormap_,
         ra0=ra0_, dec0=dec0_, radius=zoom, ns_fix=False, band=band)

    # Plot the scanned map with prior
    # Tune scale of non-scanned pixels
    llhmap_ = np.ones_like(llhmap) * np.amin(llhmap)
    idx = hp.query_disc(NSIDE_plot, *hp.ang2vec(*DecRaToThetaPhi(dec0_, ra0_)),
                        radius=zoom)
    llhmap_[idx] = llhmap[idx]
    
    # Plot range, zoomed to scan region
    ra_rng=[ra0_ - 1.2 * zoom / np.cos(dec0_),
            ra0_ + 1.2 * zoom / np.cos(dec0_)]
    dec_rng=[dec0_ - 1.2 * zoom, dec0_ + 1.2 * zoom]
    
    # Make a temp plot to get prior contours directly from the prior map
    _, _, prior_img = cartzoom(lnpriormap[0], ra_rng=ra_rng, dec_rng=dec_rng,
                               figsize=(10,8))
    plt.clf()
    _ra_grid, _dec_grid, prior_z = reconstruct_cartzoom_data(
        prior_img, ra_rng, dec_rng, smooth=4.)  
    # Normalize for proper chi2 contours
    prior_z -= np.amax(prior_z)
    levels = scs.chi2.logsf(x=[1**2, 2**2, 3**2], df=2)[::-1]
   
    # Plot the scanned region
    fig, ax, img = cartzoom(llhmap_,
                            ra_rng=[ra0_ - 1.2 * zoom / np.cos(dec0_),
                                    ra0_ + 1.2 * zoom / np.cos(dec0_)],
                            dec_rng=[dec0_ - 1.2 * zoom, dec0_ + 1.2 * zoom],
                            cmap=inferno_c, figsize=(10,8))
    fig.colorbar(img, ax=ax)
    
    # Plot contours from prior map, to see that we scanned within nsigma
    C = ax.contour(_ra_grid, _dec_grid, prior_z, levels, colors="w",
                   linestyles=[":", "--", "-"], label="1,2,3 sigma Prior")
    
    # Plot prior start {ra, dec}
    ax.plot(ra0_, dec0_, "C7*", ms=15, mew=2, mec="w", label="Start pos.")
    # Plot high res scan best pix
    scan_bf_idx = np.argmax(llhmap_)
    dec_scan_, ra_scan_ = ThetaPhiToDecRa(*hp.pix2ang(NSIDE_plot, scan_bf_idx))
    ax.plot(ra_scan_, dec_scan_, "wo", ms=15, mew=2, mec="C7", label="Scan BF")
    # Plot found seed
    ax.plot(ra_seed[src_idx], dec_seed[src_idx], "C7P", ms=10, mew=2, mec="w",
            label="Found seed")
    
    # Fit the llh with ns, ra, dec and hope that we arrive at the scan BF
    fit_res = fit_llh_ratio(x0=[ns_seed[src_idx], ra_seed[src_idx],
                                dec_seed[src_idx]], fix_ra_dec=False,
                            data=data, lnpriormap=lnpriormap_, band=band)
    # Plot best fit
    ax.plot(fit_res.x[1], fit_res.x[2], "C3X", ms=10, mew=2, mec="w",
            label="Fit")

    # Plot gradients for scan region
    idx = hp.query_disc(NSIDE_plot, radius=zoom,
                        vec=hp.ang2vec(*DecRaToThetaPhi(dec0_, ra0_))[0])
    th, phi = hp.pix2ang(NSIDE_plot, idx)
    print("src     : ", src_idx)
    print("src ra  : ", np.rad2deg(ra0_))
    print("min phi : ", np.rad2deg(np.amin(phi)))
    print("max phi : ", np.rad2deg(np.amax(phi)))
    dec, ra = ThetaPhiToDecRa(th, phi)
    ax.quiver(ra, dec, gradmaps[:, 1][idx], gradmaps[:, 2][idx], color="w")
    # Wrap to handle cases at ra ~ 0, 360. Only for plotting the grads correctly
    if ra0_ - zoom < 0:
        ra_l = ra - 2 * np.pi
        ax.quiver(ra_l, dec, gradmaps[:, 1][idx], gradmaps[:, 2][idx], color="w")
        ax.axvline(0., ls="--", c="w")
    if ra0_ + zoom > 2. * np.pi:
        ra_r = ra + 2. * np.pi
        ax.quiver(ra_r, dec, gradmaps[:, 1][idx], gradmaps[:, 2][idx], color="w")
        ax.axvline(2. * np.pi, ls="--", c="w")

    legend = ax.legend(loc="upper right")
    for legend_handle in legend.legendHandles:
        legend_handle._legmarker.set_markersize(10)
        
    # Plot title
    seed_bf = (ns_seed[src_idx], ra_seed[src_idx], dec_seed[src_idx])
    scan_bf = (nsmap[scan_bf_idx], dec_scan_[0], ra_scan_[0])
    ax.set_title("Seed: [{:.2f}, {:.2f}, {:.2f}]".format(*seed_bf) +
                 "Scan: [{:.2f}, {:.2f}, {:.2f}]".format(*scan_bf) + 
                 "Fit: [{:.2f}, {:.2f}, {:.2f}]".format(*fit_res.x))

        
    # TODO: Custom legend entries for gradients and prior contours
        
    plt.savefig("./scan_src={:d}_NSIDEseed_{:d}_NSIDEplot_{:d}.png".format(
        src_idx, NSIDE_seed, NSIDE_plot), dpi=300)
    plt.show()

## Test preselected fit vs fit with explicit box selection

Using preselection doesn't work because the LLH doesn't compensate for the BG terms anymore.

Also ns is the absolute number of expected signal eventsand if we have less background we expect more signal with so many events having SoB > 1.

In [ ]:
rnd_gen = np.random.RandomState(43374563)
nbg = int(1e5)
src_ras = np.deg2rad([40, 20, 359, 359.9, 1])
src_decs = np.deg2rad([70, 6, -80, -35, 0])
# nsig = np.arange(len(src_ras)) + 1
nsig = np.zeros_like(src_ras, dtype=int)
sig = {"nsig": nsig, "ra": src_ras, "dec": src_decs, "sigma": 0.}
ras, decs, sigmas, idx = make_sample(nbg, sig=sig, rnd=rnd_gen)

data = (ras, decs, sigmas)
box = np.deg2rad(15)

In [ ]:
# Use all events for comparison
i = 0

llh = []
ns = np.linspace(0, 10)
for nsi in ns:
    params = [nsi, src_ras[i], src_decs[i]]
    llh.append(llh_ratio(params, data, box=None)[0])
    
plt.plot(ns, llh)
plt.axvline(0, ls="--", c="C7")
plt.axhline(0, ls="--", c="C7")
plt.show()

In [ ]:
# Explicit box
i = 0

llh = []
ns = np.linspace(0, 10)
for nsi in ns:
    params = [nsi, src_ras[i], src_decs[i]]
    llh.append(llh_ratio(params, data, box=None)[0])
    
plt.plot(ns, llh)
plt.axvline(0, ls="--", c="C7")
plt.axhline(0, ls="--", c="C7")
plt.show()

In [ ]:
# Preselect and use no band
i = 0
_, data_i,_ = _select_events(params, data, box=box)

llh = []
ns = np.linspace(0, 10)
for nsi in ns:
    params = [nsi, src_ras[i], src_decs[i]]
    llh.append(llh_ratio(params, data_i, box=None)[0])
    
plt.plot(ns, llh)
plt.axvline(0, ls="--", c="C7")
plt.axhline(0, ls="--", c="C7")
plt.show()

## Test ra periodic boundaries in llh and box mode

In [ ]:
rnd_gen = np.random.RandomState(43374563)
nbg = int(1e5)
src_ras = np.deg2rad([40, 20, 359, 359.9, 1])
src_decs = np.deg2rad([70, 6, -80, -35, 0])
# nsig = np.arange(len(src_ras)) + 1
nsig = np.zeros_like(src_ras, dtype=int)
sig = {"nsig": nsig, "ra": src_ras, "dec": src_decs, "sigma": 0.}
ras, decs, sigmas, idx = make_sample(nbg, sig=sig, rnd=rnd_gen)

data = (ras, decs, sigmas)

In [ ]:
%%time
NSIDE_prior = 256  # Less resolution here, enough to test box mode
src_ths, src_phis = DecRaToThetaPhi(src_decs, src_ras)
sigma0 = np.deg2rad([1., 1., 0.7, 1., 0.5])
    
ln_priors = []
ln_priors_dphi = []
ln_priors_dth = []
for i, (th0i, phi0i, sigma0i) in enumerate(zip(src_ths, src_phis, sigma0)):
    print(u"Making prior at: RA={:.2f}°, DEC={:.2f}°, SIGMA={:.3f}°".format(
        np.rad2deg(ThetaPhiToDecRa(th0i, phi0i)[1])[0], 
        np.rad2deg(ThetaPhiToDecRa(th0i, phi0i)[0])[0], np.rad2deg(sigma0i)))
    prior = gaussian_on_a_sphere(mean_th=th0i, mean_phi=phi0i, 
                                 clip=5., NSIDE=NSIDE_prior, sigma=sigma0i)[0]
    # Avoid taking the log of 0, use the smallest valid map number instead
    prior[prior <= 0] = np.amin(prior[prior > 0])
    ln_prior = np.log(prior)

    # Make src_ras, src_decs more realistic by setting it to the map maximum
    idx = np.argmax(ln_prior)
    src_ths[i], src_phis[i] = hp.pix2ang(NSIDE_prior, idx)
    src_decs[i], src_ras[i] = ThetaPhiToDecRa(src_ths[i], src_phis[i])
    print("  New src pos at maximum of the prior map: " +
          "RA={:.2f}°, DEC={:.2f}°".format(*np.rad2deg((src_ras[i], src_decs[i]))))

    # Get the log derivatives from the finite elements method
    ln_prior_dphi, ln_prior_dth = map_der(ln_prior)
    
    ln_priors.append(ln_prior)
    ln_priors_dphi.append(ln_prior_dphi)
    ln_priors_dth.append(ln_prior_dth)
    
lnpriormap = (np.sum(ln_priors, axis=0), ln_priors_dth, ln_priors_dphi)

In [ ]:
# Iterate through srcs
i = 2
lnpriormap_ = (lnpriormap[0], [lnpriormap[1][i]], [lnpriormap[2][i]])

# For 20° it should pretty much equal the result with all events
for box_i in [None, np.deg2rad(5), np.deg2rad(10), np.deg2rad(20)]:
    print(box_i)
    for shift in [-2*np.pi, 0., 2*np.pi, 4*np.pi]:
        print("  Shift: ", shift / np.pi, " pi")
        print("  ", llh_ratio([0.2, 2*np.pi - 0.005 + shift, 0.], data,
                              lnpriormap=lnpriormap_, box=box_i))
    print("--")

## Test scan seeding function

In [ ]:
rnd_gen = np.random.RandomState(43374563)
nbg = int(1e5)
# src_ras = np.deg2rad([40, 20, 277, 300, 310])
src_ras = np.deg2rad([40, 20, 359, 359.9, 1])
src_decs = np.deg2rad([70, 6, -80, -35, 0])
# nsig = np.arange(len(src_ras)) + 1
nsig = np.zeros_like(src_ras, dtype=int)
sig = {"nsig": nsig, "ra": src_ras, "dec": src_decs, "sigma": 0.}
ras, decs, sigmas, idx = make_sample(nbg, sig=sig, rnd=rnd_gen)

data = (ras, decs, sigmas)

In [ ]:
%%time
# Construct priors for visual comparison only (see note on top)
NSIDE_prior = 512
src_ths, src_phis = DecRaToThetaPhi(src_decs, src_ras)
sigma0 = np.deg2rad([1., 1., 0.7, 1., 0.5])
    
ln_priors = []
ln_priors_dphi = []
ln_priors_dth = []
for i, (th0i, phi0i, sigma0i) in enumerate(zip(src_ths, src_phis, sigma0)):
    print(u"Making prior at: RA={:.2f}°, DEC={:.2f}°, SIGMA={:.3f}°".format(
        np.rad2deg(ThetaPhiToDecRa(th0i, phi0i)[1])[0], 
        np.rad2deg(ThetaPhiToDecRa(th0i, phi0i)[0])[0], np.rad2deg(sigma0i)))
    prior = gaussian_on_a_sphere(mean_th=th0i, mean_phi=phi0i, 
                                 clip=5., NSIDE=NSIDE_prior, sigma=sigma0i)[0]
    # Avoid taking the log of 0, use the smallest valid map number instead
    prior[prior <= 0] = np.amin(prior[prior > 0])
    ln_prior = np.log(prior)

    # Make src_ras, src_decs more realistic by setting it to the map maximum
    idx = np.argmax(ln_prior)
    src_ths[i], src_phis[i] = hp.pix2ang(NSIDE_prior, idx)
    src_decs[i], src_ras[i] = ThetaPhiToDecRa(src_ths[i], src_phis[i])
    print("  New src pos at maximum of the prior map: " +
          "RA={:.2f}°, DEC={:.2f}°".format(*np.rad2deg((src_ras[i], src_decs[i]))))

    # Get the log derivatives from the finite elements method
    ln_prior_dphi, ln_prior_dth = map_der(ln_prior)
    
    ln_priors.append(ln_prior)
    ln_priors_dphi.append(ln_prior_dphi)
    ln_priors_dth.append(ln_prior_dth)
    
lnpriormap = (np.sum(ln_priors, axis=0), ln_priors_dth, ln_priors_dphi)

Get seeds from local scans

In [ ]:
%%time
NSIDE_seed = 64
radii = 3. * sigma0
band = np.deg2rad(15)

seeds = get_seed_scan(src_ras, src_decs, radii, NSIDE_seed, data,
                      lnpriormap=lnpriormap, band=band)
ns_seed, ra_seed, dec_seed = seeds
[print(si) for si in seeds]

Scan whole region again to verify a reasonable pixel per src is found as a local seed.

In [ ]:
# %%time
NSIDE_plot = 512

for src_idx in tqdm(range(len(src_decs))[:]):
    ra0_ = src_ras[src_idx]
    dec0_ = src_decs[src_idx]
    lnpriormap_ = (lnpriormap[0], [lnpriormap[1][src_idx]],
                   [lnpriormap[2][src_idx]])
    zoom = radii[src_idx]

    # Scan with prior
    nsmap, llhmap, gradmaps = sky_scan(NSIDE=NSIDE_plot, data=data,
         lnpriormap=lnpriormap_,
         ra0=ra0_, dec0=dec0_, radius=zoom, ns_fix=False, band=band)

    # Plot the scanned map with prior
    # Tune scale of non-scanned pixels
    llhmap_ = np.ones_like(llhmap) * np.amin(llhmap)
    idx = hp.query_disc(NSIDE_plot, *hp.ang2vec(*DecRaToThetaPhi(dec0_, ra0_)),
                        radius=zoom)
    llhmap_[idx] = llhmap[idx]
    
    # Plot range, zoomed to scan region
    ra_rng=[ra0_ - 1.2 * zoom / np.cos(dec0_),
            ra0_ + 1.2 * zoom / np.cos(dec0_)]
    dec_rng=[dec0_ - 1.2 * zoom, dec0_ + 1.2 * zoom]
    
    # Make a temp plot to get prior contours directly from the prior map
    _, _, prior_img = cartzoom(lnpriormap[0], ra_rng=ra_rng, dec_rng=dec_rng,
                               figsize=(10,8))
    plt.clf()
    _ra_grid, _dec_grid, prior_z = reconstruct_cartzoom_data(
        prior_img, ra_rng, dec_rng, smooth=4.)  
    # Normalize for proper chi2 contours
    prior_z -= np.amax(prior_z)
    levels = scs.chi2.logsf(x=[1**2, 2**2, 3**2], df=2)[::-1]
   
    # Plot the scanned region
    fig, ax, img = cartzoom(llhmap_,
                            ra_rng=[ra0_ - 1.2 * zoom / np.cos(dec0_),
                                    ra0_ + 1.2 * zoom / np.cos(dec0_)],
                            dec_rng=[dec0_ - 1.2 * zoom, dec0_ + 1.2 * zoom],
                            cmap=inferno_c, figsize=(10,8))
    fig.colorbar(img, ax=ax)
    
    # Plot contours from prior map, to see that we scanned within nsigma
    C = ax.contour(_ra_grid, _dec_grid, prior_z, levels, colors="w",
                   linestyles=[":", "--", "-"], label="1,2,3 sigma Prior")
    
    # Plot prior start {ra, dec}
    ax.plot(ra0_, dec0_, "C7*", ms=15, mew=2, mec="w", label="Start pos.")
    # Plot high res scan best pix
    scan_bf_idx = np.argmax(llhmap_)
    dec_scan_, ra_scan_ = ThetaPhiToDecRa(*hp.pix2ang(NSIDE_plot, scan_bf_idx))
    ax.plot(ra_scan_, dec_scan_, "wo", ms=15, mew=2, mec="C7", label="Scan BF")
    # Plot found seed
    ax.plot(ra_seed[src_idx], dec_seed[src_idx], "C7P", ms=10, mew=2, mec="w",
            label="Found seed")
    
    # Fit the llh with ns, ra, dec and hope that we arrive at the scan BF
    fit_res = fit_llh_ratio(x0=[ns_seed[src_idx], ra_seed[src_idx],
                                dec_seed[src_idx]], fix_ra_dec=False,
                            data=data, lnpriormap=lnpriormap_, band=band)
    # Plot best fit
    ax.plot(fit_res.x[1], fit_res.x[2], "C3X", ms=10, mew=2, mec="w",
            label="Fit")

    # Plot gradients for scan region
    idx = hp.query_disc(NSIDE_plot, radius=zoom,
                        vec=hp.ang2vec(*DecRaToThetaPhi(dec0_, ra0_))[0])
    th, phi = hp.pix2ang(NSIDE_plot, idx)
    print("src     : ", src_idx)
    print("src ra  : ", np.rad2deg(ra0_))
    print("min phi : ", np.rad2deg(np.amin(phi)))
    print("max phi : ", np.rad2deg(np.amax(phi)))
    dec, ra = ThetaPhiToDecRa(th, phi)
    ax.quiver(ra, dec, gradmaps[:, 1][idx], gradmaps[:, 2][idx], color="w")
    # Wrap to handle cases at ra ~ 0, 360. Only for plotting the grads correctly
    if ra0_ - zoom < 0:
        ra_l = ra - 2 * np.pi
        ax.quiver(ra_l, dec, gradmaps[:, 1][idx], gradmaps[:, 2][idx], color="w")
        ax.axvline(0., ls="--", c="w")
    if ra0_ + zoom > 2. * np.pi:
        ra_r = ra + 2. * np.pi
        ax.quiver(ra_r, dec, gradmaps[:, 1][idx], gradmaps[:, 2][idx], color="w")
        ax.axvline(2. * np.pi, ls="--", c="w")

    legend = ax.legend(loc="upper right")
    for legend_handle in legend.legendHandles:
        legend_handle._legmarker.set_markersize(10)
        
    # Plot title
    seed_bf = (ns_seed[src_idx], ra_seed[src_idx], dec_seed[src_idx])
    scan_bf = (nsmap[scan_bf_idx], dec_scan_[0], ra_scan_[0])
    ax.set_title("Seed: [{:.2f}, {:.2f}, {:.2f}]".format(*seed_bf) +
                 "Scan: [{:.2f}, {:.2f}, {:.2f}]".format(*scan_bf) + 
                 "Fit: [{:.2f}, {:.2f}, {:.2f}]".format(*fit_res.x))

        
    # TODO: Custom legend entries for gradients and prior contours
        
    plt.savefig("./scan_src={:d}_NSIDEseed_{:d}_NSIDEplot_{:d}.png".format(
        src_idx, NSIDE_seed, NSIDE_plot), dpi=300)
    plt.show()

## Scanning to test the gradients per source location

In [ ]:
rnd_gen = np.random.RandomState(43374563)
nbg = int(1e5)
ra0 = np.deg2rad([40, 20, 277, 300, 310])
dec0 = np.deg2rad([70, 6, -80, -35, 0])
nsig = np.arange(len(ra0)) + 1
sig = {"nsig": nsig, "ra": ra0, "dec": dec0, "sigma": 0.}
ras, decs, sigmas, idx = make_sample(nbg, sig=sig, rnd=rnd_gen)

data = (ras, decs, sigmas)

In [ ]:
# Used to make many plots, when in need to recreate them ...
# np.save(arr=data, file="./pass2data.npy")

In [ ]:
%%time
# Construct priors for visula comparison only (see note on top)
NSIDE = 512
th0, phi0 = DecRaToThetaPhi(dec0, ra0)
sigma0 = np.deg2rad([1., 1., 0.7, 1., 0.5])
    
ln_priors = []
ln_priors_dphi = []
ln_priors_dth = []
for i, (th0i, phi0i, sigma0i) in enumerate(zip(th0, phi0, sigma0)):
    print(u"Making prior at: PHI={:.2f}, DEC={:.2f}, SIGMA={:.3f}".format(
        phi0i, th0i, sigma0i))
    prior = gaussian_on_a_sphere(mean_th=th0i, mean_phi=phi0i, 
                                 clip=5., NSIDE=NSIDE, sigma=sigma0i)[0]
    # Avoid taking the log of 0, use the smallest valid map number instead
    prior[prior <= 0] = np.amin(prior[prior > 0])
    ln_prior = np.log(prior)

    # Make ra0, dec0 more realistic by setting it to the map maximum as in HESE
    idx = np.argmax(ln_prior)
    th0[i], phi0[i] = hp.pix2ang(NSIDE, idx)
    dec0[i], ra0[i] = ThetaPhiToDecRa(th0[i], phi0[i])

    # Get the log derivatives from the finite elements method
    ln_prior_dphi, ln_prior_dth = map_der(ln_prior)
    
    ln_priors.append(ln_prior)
    ln_priors_dphi.append(ln_prior_dphi)
    ln_priors_dth.append(ln_prior_dth)
    
lnpriormap = (np.sum(ln_priors, axis=0), ln_priors_dth, ln_priors_dphi)

Sanning with fixed ns steps

In [ ]:
# %%time
NSIDE = 128
zoom = np.deg2rad(2)
ns_fixs = [0.5, 1., 2.5, 5., 7.5, 10., 15.]

for src_idx in range(len(dec0))[:1]:
    ra0_ = ra0[src_idx]
    dec0_ = dec0[src_idx]

    for ns_fix in tqdm(ns_fixs[:2]):
        # Scan with prior
        nsmap, llhmap, gradmaps = sky_scan(NSIDE=NSIDE, data=data,
             lnpriormap=(lnpriormap[0],
                         [lnpriormap[1][src_idx]],
                         [lnpriormap[2][src_idx]]),
             ra0=ra0_, dec0=dec0_, radius=zoom, ns_fix=ns_fix)

        # Plot the scanned map with prior
        # Tune scale of non-scanned pixels
        llhmap_ = np.ones_like(llhmap) * np.amin(llhmap)
        idx = hp.query_disc(NSIDE, *hp.ang2vec(*DecRaToThetaPhi(dec0_, ra0_)),
                            radius=zoom)
        llhmap_[idx] = llhmap[idx]

        fig, ax, img = cartzoom(llhmap_,
                                ra_rng=[ra0_ - 1.2 * zoom / np.cos(dec0_),
                                        ra0_ + 1.2 * zoom / np.cos(dec0_)],
                                dec_rng=[dec0_ - 1.2 * zoom, dec0_ + 1.2 * zoom],
                                cmap=inferno_c, figsize=(10,8))
        fig.colorbar(img, ax=ax)
        ax.plot(ra0_, dec0_, "k*", ms=15, mew=2, mec="w")

        # Plot gradients for scan region
        idx = hp.query_disc(NSIDE, radius=zoom,
                            vec=hp.ang2vec(*DecRaToThetaPhi(dec0_, ra0_))[0])
        th, phi = hp.pix2ang(NSIDE, idx)
        dec, ra = ThetaPhiToDecRa(th, phi)
        ax.quiver(ra, dec, gradmaps[:, 1][idx], gradmaps[:, 2][idx], color="w")

        plt.savefig("./scan_src={:d}_nsfix={:.2f}_prior.png".format(src_idx, ns_fix),
                    dpi=300)
        plt.show()
        
        
        # Scan w/o prior            
        nsmap_np, llhmap_np, gradmaps_np = sky_scan(NSIDE=NSIDE, data=data,
                 lnpriormap=None,
                 ra0=ra0_, dec0=dec0_, radius=zoom, ns_fix=ns_fix)

        # Plot the scanned map without prior
        # Tune scale of non-scanned pixels
        llhmap_ = np.ones_like(llhmap_np) * np.amin(llhmap_np)
        idx = hp.query_disc(NSIDE, *hp.ang2vec(*DecRaToThetaPhi(dec0_, ra0_)),
                            radius=zoom)
        llhmap_[idx] = llhmap_np[idx]

        fig, ax, img = cartzoom(llhmap_,
                                ra_rng=[ra0_ - 1.2 * zoom / np.cos(dec0_),
                                        ra0_ + 1.2 * zoom / np.cos(dec0_)],
                                dec_rng=[dec0_ - 1.2 * zoom, dec0_ + 1.2 * zoom],
                                cmap=inferno_c, figsize=(10,8))
        fig.colorbar(img, ax=ax)
        ax.plot(ra0_, dec0_, "k*", ms=15, mew=2, mec="w")

        # Plot gradients for scan region
        idx = hp.query_disc(NSIDE, radius=zoom,
                            vec=hp.ang2vec(*DecRaToThetaPhi(dec0_, ra0_))[0])
        th, phi = hp.pix2ang(NSIDE, idx)
        dec, ra = ThetaPhiToDecRa(th, phi)
        ax.quiver(ra, dec, gradmaps_np[:, 1][idx], gradmaps_np[:, 2][idx], color="w")

        plt.savefig("./scan_src={:d}_nsfix={:.2f}_noprior.png".format(src_idx, ns_fix),
                    dpi=300)
        plt.show()

Sanning with profile fitting

In [ ]:
%%time
NSIDE = 64
band = np.deg2rad(15)

for src_idx in tqdm(range(len(dec0))[:]):
    ra0_ = ra0[src_idx]
    dec0_ = dec0[src_idx]
    zoom = 3. * sigma0[src_idx]

    # Scan with prior
    nsmap, llhmap, gradmaps = sky_scan(NSIDE=NSIDE, data=data,
         lnpriormap=(lnpriormap[0],
                     [lnpriormap[1][src_idx]],
                     [lnpriormap[2][src_idx]]),
         ra0=ra0_, dec0=dec0_, radius=zoom, ns_fix=False, band=band)

    # Plot the scanned map with prior
    # Tune scale of non-scanned pixels
    llhmap_ = np.ones_like(llhmap) * np.amin(llhmap)
    idx = hp.query_disc(NSIDE, *hp.ang2vec(*DecRaToThetaPhi(dec0_, ra0_)),
                        radius=zoom)
    llhmap_[idx] = llhmap[idx]

    fig, ax, img = cartzoom(llhmap_,
                            ra_rng=[ra0_ - 1.2 * zoom / np.cos(dec0_),
                                    ra0_ + 1.2 * zoom / np.cos(dec0_)],
                            dec_rng=[dec0_ - 1.2 * zoom, dec0_ + 1.2 * zoom],
                            cmap=inferno_c, figsize=(10,8))
    fig.colorbar(img, ax=ax)
    ax.plot(ra0_, dec0_, "k*", ms=15, mew=2, mec="w")

    # Plot gradients for scan region
    idx = hp.query_disc(NSIDE, radius=zoom,
                        vec=hp.ang2vec(*DecRaToThetaPhi(dec0_, ra0_))[0])
    th, phi = hp.pix2ang(NSIDE, idx)
    dec, ra = ThetaPhiToDecRa(th, phi)
    ax.quiver(ra, dec, gradmaps[:, 1][idx], gradmaps[:, 2][idx], color="w")

    plt.savefig("./scan_src={:d}_nsfix={:.2f}_prior.png".format(src_idx, ns_fix),
                dpi=300)
    plt.show()


    # Scan w/o prior            
    nsmap_np, llhmap_np, gradmaps_np = sky_scan(NSIDE=NSIDE, data=data,
             lnpriormap=None,
             ra0=ra0_, dec0=dec0_, radius=zoom, ns_fix=False, band=band)

    # Plot the scanned map without prior
    # Tune scale of non-scanned pixels
    llhmap_ = np.ones_like(llhmap_np) * np.amin(llhmap_np)
    idx = hp.query_disc(NSIDE, *hp.ang2vec(*DecRaToThetaPhi(dec0_, ra0_)),
                        radius=zoom)
    llhmap_[idx] = llhmap_np[idx]

    fig, ax, img = cartzoom(llhmap_,
                            ra_rng=[ra0_ - 1.2 * zoom / np.cos(dec0_),
                                    ra0_ + 1.2 * zoom / np.cos(dec0_)],
                            dec_rng=[dec0_ - 1.2 * zoom, dec0_ + 1.2 * zoom],
                            cmap=inferno_c, figsize=(10,8))
    fig.colorbar(img, ax=ax)
    ax.plot(ra0_, dec0_, "k*", ms=15, mew=2, mec="w")

    # Plot gradients for scan region
    idx = hp.query_disc(NSIDE, radius=zoom,
                        vec=hp.ang2vec(*DecRaToThetaPhi(dec0_, ra0_))[0])
    th, phi = hp.pix2ang(NSIDE, idx)
    dec, ra = ThetaPhiToDecRa(th, phi)
    ax.quiver(ra, dec, gradmaps_np[:, 1][idx], gradmaps_np[:, 2][idx], color="w")

    plt.savefig("./scan_src={:d}_nsfix={:.2f}_noprior.png".format(src_idx, ns_fix),
                dpi=300)
    plt.show()

In [ ]:
# Scan ns to check correct scales in ra, dec scan
src_idx = 0
lnpriormap_ =(lnpriormap[0], [lnpriormap[1][src_idx]], [lnpriormap[2][src_idx]])
lnpriormap_ = None

x0 = [0., ra0[src_idx], dec0[src_idx]]

ns_ = np.linspace(0, 5, 100)
TS = []
ns_grad = []
for nsi in ns_:
    x0[0] = nsi
    TSi, gradi = llh_ratio(params=x0, data=data_i, lnpriormap=lnpriormap_,
                           band=np.deg2rad(15))
    TS.append(TSi)
    ns_grad.append(gradi[0])

In [ ]:
plt.plot(ns_, TS)
plt.plot(ns_, ns_grad)
mids = .5 * (ns_[:-1] + ns_[1:])
plt.plot(mids, np.diff(TS)/np.diff(ns_), ls="--", c="k")

plt.axhline(0, c="C7")
plt.axvline(0, c="C7")

plt.show()

## Test band selection mode

Note: using naive box mode close to the poles is a very bad idea, because events are contributing for almost all RA values, because of the small solid angle.

Use band mode to be safe or a true circle mode, selecting within a circle respecting the behaviour on a sphere.
Circle mode would need to calc all angular distances, so we wouldn't get a computational advantage.

In [ ]:
rnd_gen = np.random.RandomState(43374563)
nbg = int(1e5)
nsig = 5
src_ras = np.deg2rad([40., 20., 359., 359.9, 1.])
src_decs = np.deg2rad([10., -30., -81., 85., 30.])
sig = {"nsig": nsig, "ra": src_ras, "dec": src_decs, "sigma": 0.}
ras, decs, sigmas, idx = make_sample(nbg, sig=sig, rnd=rnd_gen)

In [ ]:
ns0 = [3., 6., 3.2, 5., 2.]
x = (ns0, src_ras, src_decs)
box = len(ns0) * [np.deg2rad(15)]
data = (ras, decs, sigmas)

print("All")
print(llh_ratio(x, data, box=None))

print("")
print("Band")
print(llh_ratio(x, data, box=box))

In [ ]:
# Preselect events and calc SoBs
_, (ras_, decs_, sigmas_), _ = _select_events(
    (len(src_ras) * [-1], src_ras, src_decs), data=data, box=box)
pdf_s, grad_s = pdf_sig(src_decs=src_decs, src_ras=src_ras,
                        ras=ras_, decs=decs_, sigmas=sigmas_)
pdf_b, grad_b = pdf_bg(src_decs=src_decs, decs=decs_)
pdf_sb, grad_sb = pdf_sob(src_decs=src_decs, src_ras=src_ras,
                          ras=ras_, decs=decs_, sigmas=sigmas_)
assert np.allclose(pdf_sb, pdf_s / pdf_b)

for src_idx in range(len(src_decs)):
    pdf_sbi = pdf_sb[src_idx]
    src_rai, src_deci = src_ras[src_idx], src_decs[src_idx]

    # Only show SoB > 1 in colors
    m = (pdf_sbi > 1.)
    # Scale sigmas for better visibility
    scale = 20

    fig, ax = plt.subplots(1, 1, figsize=(7, 6))
    
    # Plot selection box - Code copied from _selecrt_events function
    boxi = box[src_idx] / 2.
    top_height = np.abs(src_deci) + boxi
    if top_height > np.deg2rad(89):
        box_width = np.pi
    else:
        box_width = boxi / np.cos(top_height)
    
    box_ras = [src_rai - box_width, src_rai - box_width,
                src_rai + box_width, src_rai + box_width,
                src_rai - box_width]
    box_decs = [src_deci - boxi, src_deci + boxi,
                 src_deci + boxi, src_deci - boxi,
                 src_deci - boxi]
    ax.fill_between(box_ras, box_decs, alpha=.1, color="C3")
    ax.plot(box_ras, box_decs, alpha=.5, color="C3")

    # Boundaries
    ral = np.amin(box_ras) - np.deg2rad(2)
    rar = np.amax(box_ras) + np.deg2rad(2)
    decl = np.amin(box_decs) - np.deg2rad(2)
    dech = np.amax(box_decs) + np.deg2rad(2)

    # Events
    ax.scatter(ras_[~m], decs_[~m], s=scale * np.rad2deg(sigmas_[~m]), c="C7",
               alpha=0.6)
    ax.scatter(ras_[m], decs_[m], s=scale * np.rad2deg(sigmas_[m]),
               c=np.log10(pdf_sbi[m]))
    
    # Repeat to wrap
    if ral < 0:
        ax.axvline(0, ls="--", c="C7")
        ras_neg = ras_ - 2 * np.pi
        ax.scatter(ras_neg[~m], decs_[~m], s=scale * np.rad2deg(sigmas_[~m]),
                   c="C7", alpha=0.6)
        ax.scatter(ras_neg[m], decs_[m], s=scale * np.rad2deg(sigmas_[m]),
                   c=np.log10(pdf_sbi[m]))
    elif rar > 2. * np.pi:
        ax.axvline(2. * np.pi, ls="--", c="C7")
        ras_pos = ras_ + 2 * np.pi
        ax.scatter(ras_pos[~m], decs_[~m], s=scale * np.rad2deg(sigmas_[~m]),
                   c="C7", alpha=0.6)
        ax.scatter(ras_pos[m], decs_[m], s=scale * np.rad2deg(sigmas_[m]),
                   c=np.log10(pdf_sbi[m]))
        
    # Source center
    ax.plot(src_rai, src_deci, "r*", ms=10)

    ax.set_xlim(ral, rar)
    ax.set_ylim(decl, dech)

    make_astro_xaxis(ax)

    plt.show()

## Show source variation for multiple sources and priors

Respect source variation according to a gaussian prior for signal injection.

Here the injection draws from a gaussian, in reality we would draw from the healpy prior map instead.
Here it doesn't make a difference.

In [ ]:
%%time
# Construct priors for visula comparison only (see note on top)
NSIDE = 256
ra0 = np.deg2rad([120, 150, 300])
dec0 = np.deg2rad([-10, 20, 25])  # True src positions
th0, phi0 = DecRaToThetaPhi(dec0, ra0)
sigma0 = np.deg2rad([0.5, 1., 2.])
    
ln_priors = []
ln_priors_dphi = []
ln_priors_dth = []
for i, (th0i, phi0i, sigma0i) in enumerate(zip(th0, phi0, sigma0)):
    print(u"Making prior at: PHI={:.2f}, DEC={:.2f}, SIGMA={:.3f}".format(
        phi0i, th0i, sigma0i))
    prior = gaussian_on_a_sphere(mean_th=th0i, mean_phi=phi0i, 
                                 clip=5., NSIDE=NSIDE, sigma=sigma0i)[0]
    # Avoid taking the log of 0, use the smallest valid map number instead
    prior[prior <= 0] = np.amin(prior[prior > 0])
    ln_prior = np.log(prior)

    # Make ra0, dec0 more realistic by setting it to the map maximum as in HESE
    idx = np.argmax(ln_prior)
    th0[i], phi0[i] = hp.pix2ang(NSIDE, idx)
    dec0[i], ra0[i] = ThetaPhiToDecRa(th0[i], phi0[i])

    # Get the log derivatives from the finite elements method
    ln_prior_dphi, ln_prior_dth = map_der(ln_prior)
    
    ln_priors.append(ln_prior)
    ln_priors_dphi.append(ln_prior_dphi)
    ln_priors_dth.append(ln_prior_dth)

In [ ]:
rnd_gen = np.random.RandomState(238823442)

zoom = np.deg2rad(5)
sig_scale = 5

for i in range(10):
    # Create a test sample
    nbg = int(1e5)
    sig = {"nsig": 100, "ra": ra0, "dec": dec0, "sigma": sigma0}

    ras, decs, sigmas, ev_idx = make_sample(nbg=nbg, sig=sig, rnd=rnd_gen)

    data = (ras, decs, sigmas)
    lnpriormap = [(lnp, lnp_dth, lnp_dphi) for lnp, lnp_dth, lnp_dphi in
                  zip(ln_priors, ln_priors_dth, ln_priors_dphi)]

    # All srcs on display
    fig, axs = plt.subplots(1, 3, figsize=(18, 6))
    for src_idx, axi in enumerate(axs):
        ra0_ , dec0_ = ra0[src_idx], dec0[src_idx]
        _, _, img = cartzoom_radius(ln_priors[src_idx], r=zoom, ax=axi,
                                    center=[ra0_, dec0_], cmap="gray")
        fig.colorbar(img, ax=ax)

        ev_m = (((ras > ra0_ - zoom) & (ras < ra0_ + zoom)) &
                ((decs > dec0_ - zoom) & (decs < dec0_ + zoom)) &
                (ev_idx == 0))
        axi.scatter(ras[ev_m], decs[ev_m],
                    s=sig_scale * np.rad2deg(sigmas[ev_m]),
                    c="C0", alpha=0.7)

        # Src events in red
        m = (src_idx + 1 == ev_idx)
        axi.scatter(ras[m], decs[m],
                    s=sig_scale * np.rad2deg(sigmas[ev_m]), c="r")
        # Approx. new src position
        axi.plot(np.mean(ras[m]), np.mean(decs[m]), "g*", ms=7)
        # Best fit original src position
        axi.plot(ra0[src_idx], dec0[src_idx], "gd", ms=7)
        
        axi.set_title("src {:d}. Prior {:.1f} deg".format(src_idx,
                                                          np.rad2deg(sigma0i)))
        make_astro_xaxis(ax)

    fig.tight_layout()
    plt.savefig("./multi_src_inj_{:02d}.png".format(i), dpi=150)
    plt.clf()

In [ ]:
fig, ax, img = cartzoom(np.sum(ln_priors, axis=0))
fig.colorbar(img, ax=ax)

ax.scatter(ras, decs, s=5*sigmas, c="w", marker=".", alpha=0.5)
ax.scatter(ras[sig_idx > 0], decs[sig_idx > 0], s=5*sigmas[sig_idx > 0], c="r",
           marker=".", alpha=0.75)

make_astro_xaxis(ax)
plt.tight_layout()
plt.savefig("./multi_src_injection_allsky.png", dpi=300)
plt.show()

## Show prior gradient map interpolation close to the best fit position

In [ ]:
# Place prior at the best fit src, with same sigma0 as above
NSIDE = 512

# True src position. Try various declinations.
ra0, dec0 = np.deg2rad([300]), np.deg2rad([-70])
th0, phi0 = DecRaToThetaPhi(dec0, ra0)

# Prior width
sigma0 = np.deg2rad([1])

prior = gaussian_on_a_sphere(mean_th=th0[0], mean_phi=phi0[0], clip=5.,
                             NSIDE=NSIDE, sigma=sigma0[0])[0]

prior[prior <= 0] = np.amin(prior[prior > 0])
ln_prior = np.log(prior)

# Get the log-derivatives from the finite elements method
ln_prior_dphi, ln_prior_dth = map_der(ln_prior)

# Make ra0, dec0 more realistic by setting it to the map maximum as in HESE
bf_idx = np.argmax(ln_prior)
th0, phi0 = hp.pix2ang(NSIDE, bf_idx)
dec0, ra0 = ThetaPhiToDecRa(th0, phi0)

Interpolate constructed prior gradient maps close to the best fit region

In [ ]:
ra = np.linspace(ra0 - np.deg2rad(1), ra0 + np.deg2rad(1), 100)
dec = np.linspace(dec0 - np.deg2rad(1), dec0 + np.deg2rad(1), 100)

RA, DEC = np.meshgrid(ra, dec)
RRA, DDEC = map(np.ravel, [RA, DEC])
TTH, PPHI = DecRaToThetaPhi(DDEC, RRA)

ZZth = hp.get_interp_val(ln_prior_dth, TTH, PPHI).reshape(RA.shape)
ZZphi = hp.get_interp_val(ln_prior_dphi, TTH, PPHI).reshape(RA.shape)

fig, (axl, axr) = plt.subplots(1, 2, figsize=(12, 5))

img = axl.pcolormesh(RA, DEC, ZZphi, cmap="seismic")
fig.colorbar(img, ax=axl)
axl.plot(ra0, dec0, "k*")
axl.set_title("Gradient in phi")

img = axr.pcolormesh(RA, DEC, ZZth, cmap="seismic")
fig.colorbar(img, ax=axr)
axr.plot(ra0, dec0, "k*")
axr.set_title("Gradient in theta")

plt.tight_layout()
plt.show()

Just the maps, no interpolation

In [ ]:
fig, (axl, axr) = plt.subplots(1, 2, figsize=(12, 5))

_, _, img = cartzoom_radius(ln_prior_dphi, center=[ra0, dec0], ax=axl,
                            r=np.deg2rad(2), cmap="seismic")
fig.colorbar(img, ax=axl)
axl.plot(ra0, dec0, "k*")
axl.set_title("Gradient in phi")

_, _, img = cartzoom_radius(ln_prior_dth, center=[ra0, dec0], ax=axr,
                            r=np.deg2rad(2), cmap="seismic")
fig.colorbar(img, ax=axr)
axr.plot(ra0, dec0, "k*")
axr.set_title("Gradient in theta")

plt.show()

## Compare prior map finite element gradient to alm expansion gradients

In [ ]:
# Create a test sample
rnd_gen = np.random.RandomState(23453453)
nbg = 10000
nsig = 10
# True src position 
# Note: If dec=0, than the comparison to the simple analytic gradient is correct
ra0, dec0 = np.deg2rad([300]), np.deg2rad([0])
th0, phi0 = DecRaToThetaPhi(dec0, ra0)
# Prior width (used in injection too!)
sigma0 = np.deg2rad([5])

sig = {"nsig": nsig, "ra": ra0, "dec": dec0, "sigma": sigma0}

ras, decs, sigmas, ev_idx = make_sample(nbg, sig, rnd=rnd_gen)

In [ ]:
# Place prior at the best fit src, with same sigma0 as above
NSIDE = 256
prior = gaussian_on_a_sphere(mean_th=th0[0], mean_phi=phi0[0], clip=5.,
                             NSIDE=NSIDE, sigma=sigma0[0])[0]

prior[prior <= 0] = np.amin(prior[prior > 0])
ln_prior = np.log(prior)

# Get the log-derivatives from the finite elements method
ln_prior_dphi, ln_prior_dth = map_der(ln_prior)

# And from the alm epxansion to compare against
ln_alm = hp.map2alm(ln_prior)
_, ln_prior_alm_dth, ln_prior_alm_dphi = hp.alm2map_der1(alm=ln_alm, nside=NSIDE)

Compare dphi prior to analytic solution.
ln(gaus) is a parabola, so at the horizon (euclidean regime) we see the expected linear gradient.

In [ ]:
th = th0
phis = np.linspace(0, 2*np.pi, 100)
vals = hp.get_interp_val(m=ln_prior_dphi, theta=th, phi=phis)

clip = 6
m = (phis > phi0 - clip * sigma0) & (phis < phi0 + clip * sigma0)
comp = - 1. / sigma0**2 * (phis - phi0)

plt.plot(phis[m], vals[m], c="k", lw=2.5)
plt.plot(phis[m], comp[m], c="C3", ls="--", lw=2.5)
plt.show()

In [ ]:
cmap = plt.cm.coolwarm
cmap.set_under("w")
hp.mollview(ln_prior_dphi, cmap=cmap)
hp.graticule(verbose=False)
plt.show()

Compare finite element gradients with the alm expansion gradients from healpy.

In [ ]:
cmap = plt.cm.coolwarm
cmap.set_under("w")
hp.cartview(ln_prior_dth, cmap=cmap, unit="fe grad th")
hp.projplot(th0, phi0, "k*")
hp.graticule(verbose=False)
plt.show()

hp.cartview(ln_prior_dphi, cmap=cmap, unit="fe grad phi")
hp.projplot(th0, phi0, "k*")
hp.graticule(verbose=False)
plt.show()

# alm gradients, showing artifacts near th poles
cmap = plt.cm.coolwarm
cmap.set_under("w")
hp.cartview(ln_prior_alm_dth, cmap=cmap, unit="alm grad th")
hp.projplot(th0, phi0, "k*")
hp.graticule(verbose=False)
plt.show()

hp.cartview(ln_prior_alm_dphi, cmap=cmap, unit="alm grad phi")
hp.projplot(th0, phi0, "k*")
hp.graticule(verbose=False)
plt.show()

## Gradient in ns, ra, dec

Testing the gradients for multi src cases and fitting each weight.

In [ ]:
rnd_gen = np.random.RandomState(32423)
nbg = int(1e5)
nsig = [6, 3]

ra0 = np.deg2rad([40, 250])
dec0 = np.deg2rad([-20, 80])
band = np.deg2rad(15)  # To speed up calculations

sig = {"nsig": nsig, "ra": ra0, "dec": dec0, "sigma": 0.}
ras, decs, sigmas, ev_idx = make_sample(nbg, sig=sig, rnd=rnd_gen)

In [ ]:
# Testrun to see if band is large enough
ns0 = [6., 3.]

x = (ns0, ra0, dec0)
data = (ras, decs, sigmas)

print(llh_ratio(x, data, band=None))
print(llh_ratio(x, data, band=band))

Scan in single ns for fixed ra, dec

In [ ]:
assert len(nsig) == len(dec0) == len(ra0) == 2

data = (ras, decs, sigmas)
ns = np.linspace(0, 10, 100)
for src_idx in range(len(dec0)):
    llh, grad_ns = [], []
    for nsi in ns:
        llhi, gradi = llh_ratio((nsi, ra0[src_idx], dec0[src_idx]), data,
                                band=band)
        llh.append(llhi)
        grad_ns.append(gradi[0])

    idx = np.argmax(llh)
    ns_bf = ns[idx]

    plt.plot(ns, llh)
    plt.plot(ns, grad_ns)
    plt.plot(0.5 * (ns[:-1] + ns[1:]), np.diff(llh) / np.diff(ns),
             ls=":", c="k")
    plt.axhline(0, 0, 1, ls="-", c="C7")
    plt.axvline(0, 0, 1, ls="-", c="C7")
    plt.axvline(nsig[src_idx], 0, 1, ls="-.", c="C7", label="inj nsig")
    plt.axvline(ns_bf, 0, 1, ls="--", c="C7", label="Scan bf ns")
    plt.title("grad = {:.3f}".format(grad_ns[idx]))
    plt.legend()
    plt.savefig("./ns{:d}_grad_2srcs.png".format(src_idx), dpi=150)
    plt.show()

Scan in double ns for fixed {ras, decs}, two sources

In [ ]:
data = (ras, decs, sigmas)

ns0 = np.linspace(0, 10, 20)
ns1 = np.linspace(0, 10, 20)
assert len(nsig) == len(dec0) == len(ra0) == 2

NS0, NS1 = np.meshgrid(ns0, ns1)

llh, grad_ns = [], []

# Change to test a position ([1, 2]) with no signal instead
ra0_ = ra0
ra0_ = [1., 2.]

for (ns0i, ns1i) in tqdm(zip(*map(np.ravel, [NS0, NS1]))):
    llhi, gradi = llh_ratio(([ns0i, ns1i], ra0_, dec0), data, band=band)
    llh.append(llhi)
    grad_ns.append(gradi[:2])
    
llh = np.array(llh).reshape(NS0.shape)
grad_ns0 = np.array(grad_ns)[:, 0].reshape(NS0.shape)
grad_ns1 = np.array(grad_ns)[:, 1].reshape(NS1.shape)

In [ ]:
src_idx = 1
m = (ev_idx == src_idx + 1)
scale = 300

plt.scatter(ras, decs, s=scale * sigmas, c="C7")
plt.scatter(ras[m], decs[m], s=scale * sigmas, c="C3")

plt.plot(ra0_[src_idx], dec0[src_idx], "k*", ms=15, lw=2, mfc="none")

plt.xlim(ra0_[src_idx] - 0.2, ra0_[src_idx] + 0.2)
plt.ylim(dec0[src_idx] - 0.2, dec0[src_idx] + 0.2)
make_astro_xaxis(plt.gca())
plt.show()

In [ ]:
# ra0_ = [1, 2] gradients should point to [0, 0]
img = plt.pcolormesh(NS0, NS1, llh, cmap=inferno_c)
plt.colorbar(img)

plt.quiver(NS0, NS1, grad_ns0, grad_ns1, color="w")
plt.savefig("./ns_grad_2srcs_offsrc.png", dpi=300)
plt.show()

Scan in ra, dec for fixed {ns} to test ra, dec gradients

In [ ]:
assert len(nsig) == len(dec0) == len(ra0) == 2

# Try both src locations
src_idx = 1
ra0_ = ra0[src_idx]
dec0_ = dec0[src_idx]

box_w = np.deg2rad(5)
ra_ = np.linspace(ra0_ - box_w, ra0_ + box_w, 40)
dec_ = np.linspace(dec0_ - box_w, dec0_ + box_w, 40)

RR, DD = np.meshgrid(ra_, dec_)
rr, dd = map(np.ravel, [RR, DD])

llh, grad_ra, grad_dec = [], [], []
# Temp copies for the arguments per scan point
ra0_ins = np.copy(ra0)
dec0_ins = np.copy(dec0)
for ri, di in tqdm(zip(rr, dd)):
    ra0_ins[src_idx] = ri
    dec0_ins[src_idx] = di
    llhi, gradi = llh_ratio((nsig, ra0_ins, dec0_ins), data, band=band)
    llh.append(llhi)
    grad_ra.append(gradi[2 + src_idx])
    grad_dec.append(gradi[4 + src_idx])
    
llh = np.array(llh).reshape(RR.shape)
grad_ra = np.array(grad_ra).reshape(RR.shape)
grad_dec = np.array(grad_dec).reshape(RR.shape)

In [ ]:
img = plt.pcolormesh(RR, DD, llh, cmap=inferno_c)
plt.colorbar(img)

plt.xlim(*ra_[[0, -1]])
plt.ylim(*dec_[[0, -1]])
make_astro_xaxis(plt.gca())

# Plot -grad_ra, because astro axis is from right to left in RA
plt.quiver(RR, DD, -grad_ra, grad_dec, color="w")

ev_m = (((ras > ra_[0]) & (ras < ra_[-1])) &
        ((decs > dec_[0]) & (decs < dec_[-1])))
plt.scatter(ras[ev_m], decs[ev_m], s=1000 * sigmas[ev_m], alpha=0.5,
            facecolors="none", edgecolors="w")

plt.savefig("./ra_dec_grad_2srcs_src_{:d}.png".format(src_idx), dpi=300)
plt.show()

## Event injection

In [ ]:
rnd_gen = np.random.RandomState(324243565)

In [ ]:
# Tune rayleigh to draw sigmas from
x = np.linspace(0, 10, 500)
loc = 0.3
y = scs.rayleigh.pdf(x, loc=loc, scale=0.7)
plt.hist(scs.rayleigh.rvs(loc=loc, scale=0.7, size=10000, random_state=rnd_gen),
         bins=20, normed=True, alpha=.5)
# Test manual movement, numpy only has the scale parameter
plt.hist(rnd_gen.rayleigh(scale=0.7, size=10000) + loc, bins=20,
         normed=True, alpha=.5)
plt.axvline(1, 0, 1, ls="--", color="C7")

plt.plot(x, y)
plt.xlim(0, 5)
plt.xlabel("sigma in deg")
plt.ylabel("PDF")
plt.show()

In [ ]:
# Test signal injection
sigmas = get_evt_sigmas(100, rnd_gen) * 10
ra0, dec0 = np.pi, 0.3
ra, dec = inject_signal(100, src_ra=ra0, src_dec=dec0, sigma=sigmas,
                        rnd=rnd_gen)
plt.scatter(ra, dec, s=sigmas * 100)
plt.plot(ra0, dec0, "k*", mfc="w", ms=15)
plt.xlim(0, 2*np.pi)
plt.ylim(-np.pi/2, np.pi/2)
plt.xlabel("ra in rad")
plt.ylabel("dec in rad")
plt.title("sigma scaled up for viewing")
plt.show()

In [ ]:
# Test injection, no src variation
nbg = 10000
sig = {"nsig": [10, 20, 30],
       "ra": np.deg2rad([20, 50, 300]),
       "dec": np.deg2rad([-20, 10, 50]),
       "sigma": np.deg2rad([0.])}

ev_ras, ev_decs, ev_sigmas, idx = make_sample(nbg, sig, rnd=rnd_gen)
ev_ths, ev_phis = DecRaToThetaPhi(ev_decs, ev_ras)

hp.mollview()
hp.graticule(verbose=False)
for src_idx in np.sort(np.unique(idx)):
    m = (idx == src_idx)
    if src_idx == 0:
        c = "C7"
        label = "BG"
    else:
        c = "C{:d}".format(src_idx)
        label = "src {:d}".format(src_idx)
        src_th, src_phi = DecRaToThetaPhi(sig["dec"][src_idx-1],
                                          sig["ra"][src_idx-1])
        hp.projplot(src_th, src_phi, "ko", mfc="none", lw=1.5, ms=10)
        
    hp.projscatter(ev_ths[m], ev_phis[m], s=ev_sigmas[m] * 100, c=c,
                   label=label)

plt.text(s="0h", x=2.05, y=0, horizontalalignment="left",
         verticalalignment="center")
plt.text(s="24h", x=-2.05, y=0, horizontalalignment="right",
         verticalalignment="center")
plt.text(s=u"-90°", x=0, y=-1.05, horizontalalignment="center",
         verticalalignment="top")
plt.legend()
plt.show()

## Generic -- Proper RA wrapped box mode timing psLLH vs naive

Naive timing is better by a factor of 4, because it doesn't need mod and abs operations, but simply compares to two more places at -2pi and 2pi after centering to the src positions.

Note: Execute withour timeit first to get variable names.

In [ ]:
rnd_gen = np.random.RandomState(43374563)
nbg = int(1e3)
src_ras = np.deg2rad([1, 2, 3, 10, 20, 100, 200, 300, 350, 355, 359, ])
src_decs = np.ones_like(src_ras)
ras, _, _, _ = make_sample(nbg, sig=None, rnd=rnd_gen)

# Tune to see effects
box_w = np.deg2rad(20)

In [ ]:
# %%timeit
# Select using wrapping and testing half the box on absolute values
rngs1a = (np.fabs(np.mod(ras[None, :] - src_ras[:, None] + np.pi,
                         2. * np.pi) - np.pi) < 0.5 * box_w)

for i, m in enumerate(rngs1a):
    plt.vlines(ras[m], ymin=i, ymax=i+1, linestyles="-",
               colors="C{:d}".format(i % 10))

plt.show()

In [ ]:
# %%timeit
# Same but with loop as in skylab
box2 = 0.5 * box_w

rngs1b = np.zeros((len(src_ras), len(ras)), dtype=bool)
for i, src_rai in enumerate(src_ras):
    rngs1b[i] = (np.fabs(np.mod(ras - src_rai + np.pi, 2. * np.pi) - np.pi) <
                 box2)

for i, m in enumerate(rngs1b):
    plt.vlines(ras[m], ymin=i, ymax=i+1, linestyles="-",
               colors="C{:d}".format(i % 10))

plt.show()

In [ ]:
# %%timeit
# Select using shift and testing symetrical in negative values too
ras_ = (ras[None, :] - src_ras[:, None])
rngs2a = (((ras_ > -0.5 * box_w) & (ras_ < 0.5 * box_w)) |
          ((ras_ > 2*np.pi - 0.5 * box_w) | (ras_ < -2*np.pi + 0.5 * box_w)))

for i, m in enumerate(rngs2a):
    plt.vlines(ras[m], ymin=i, ymax=i+1, linestyles="-",
               colors="C{:d}".format(i % 10))

plt.show()

In [ ]:
# %%timeit
# Same but with loop, save some memory if only m_tot is stored
box2 = 0.5 * box_w

rngs2b = np.zeros((len(src_ras), len(ras)), dtype=bool)
for i, src_rai in enumerate(src_ras):
    ras_ = ras - src_rai
    rngs2b[i] = (((ras_ > -box2) & (ras_ < box2)) |
                ((ras_ > 2. * np.pi - box2) | (ras_ < -2. * np.pi + box2)))

for i, mi in enumerate(rngs2b):
    plt.vlines(ras[mi], ymin=i, ymax=i+1, linestyles="-",
               colors="C{:d}".format(i % 10))

plt.show()

In [ ]:
# Check masks are equal
print("All masks equal: ", (np.all(rngs1a == rngs1b) &
                            np.all(rngs2a == rngs2b) &
                            np.all(rngs1a == rngs2a)))

## Generic -- Single ns LLH test: Is the best fit pixel always the largest LLH over all ns?

No it isn't, LLH functions can crossover.
This makes it very hard to find a valid seed in the multi local minima landscape when fitting multiple src positions.
In principle every pixel combination for each src and the best total ns must be tested then.
This is exponentially complec and not doable.

So fitting each srcs ns weight increses the parameter space, but maybe makes it easier to seed, because it reduces the ns correlation.

It is more model independent anyway...

In [ ]:
def logtest(ns, sobs):
    sobs = sobs[:, None]
    return np.sum(np.log(ns / len(sobs) * (sobs - 1.) + 1.), axis=0)

In [ ]:
sobs_1 = np.concatenate((np.random.uniform(0, 0.1, size=100), [100.]))
sobs_2 = np.concatenate((np.random.uniform(0, 0.1, size=100), [75., 75.]))
sobs_3 = np.concatenate((np.random.uniform(0, 0.1, size=100), [50., 50., 50.]))
sobs_4 = np.random.uniform(0, 1, size=100)

ns = np.linspace(0, 10, 200)

plt.plot(ns, logtest(ns, sobs_1), label="sobs 1")
plt.plot(ns, logtest(ns, sobs_2), ls=":", label="sobs 2")
plt.plot(ns, logtest(ns, sobs_3), ls="--", label="sobs 3")
plt.plot(ns, logtest(ns, sobs_4), ls="-.", label="sobs 4")
plt.legend()
plt.show()

## Generic -- Fitting over periodic variables with ana and fe gradients

In [ ]:
def _wrap_ang(x):
    """
    Angle wrapped periodically in [0, 2pi].
    """
    return np.mod(x - 2. * np.pi, 2. * np.pi)

def _func(x):
    """
    Sinus is a good model function. True minimum at 3/2*pi in [0,2pi].
    If seeded left of maximum at pi/2 we get stuck at zero if we dont wrap.
    """
    return np.sin(x), np.cos(x)

def fitfunc_anagrad(x):
    """
    Simple parabole with analytic gradient to test periodic wrapped fit.
    """
    # Wrap angle, should be OK with analytic gradient
    x = _wrap_ang(x)
    return _func(x)

def fitfunc_nograd(x):
    """
    Simple parabole with analytic gradient to test periodic wrapped fit.
    """
    # Wrap angle, should crash finite element gradient
    x = _wrap_ang(x)
    return _func(x)[0]

In [ ]:
# True minimum at -1
x0 = [1.]
bounds = [[-2. * np.pi, 4. * np.pi]]

ana_res = sco.minimize(fitfunc_anagrad, x0=x0, bounds=bounds, jac=True)
nog_res = sco.minimize(fitfunc_nograd, x0=x0, bounds=bounds, jac=False)

In [ ]:
ana_res

In [ ]:
nog_res

In [ ]:
x = np.linspace(bounds[0][0], bounds[0][1], 300)
func, grad = fitfunc_anagrad(x)

plt.plot(x, func)
plt.plot(x, grad)

plt.axhline(0, ls="-", c="C7")
plt.axvline(0, ls="-", c="C7")
plt.axvline(2*np.pi, ls="-", c="C7")
plt.axvline(x[0], ls="--", c="C7")
plt.axvline(x[-1], ls="--", c="C7")

plt.axvline(3.*np.pi/2., ls="-.", c="C7")
plt.axvline(ana_res.x, ls="-.", c="C4")
plt.axvline(nog_res.x, ls="-.", c="C5")

plt.show()

In [ ]:
# Small test for the wrapper function
ras_ = np.linspace(-2*np.pi, 4*np.pi, 20)
ras__ = _wrap_ang(ras_)
print(ras_)
print(ras__)

plt.plot(ras_, ras_)
plt.plot(ras_, ras__)
plt.show()

## Generic -- Numerical 1st derivate test

In [ ]:
phis = np.arange(5)**2
# Make periodic boundaries
phis = np.r_[phis, phis[::-1]]
phis = np.r_[phis[-1], phis, phis[0]]

dphis = (phis[2:] - phis[:-2]) / 2

x = np.arange(len(phis) - 2)
mids = 0.5 * (x[:-1] + x[1:])

# Plot symmetric, forward and backward derivative
plt.plot(x, phis[1:-1], marker="o", label="fun")
plt.plot(x, dphis, marker="o", label="sym grad")
plt.plot(mids, np.diff(phis[1:-1]), marker="o", label="forw grad")
plt.plot(mids, np.diff(phis[1:-1]), marker="o", label="back grad")

plt.axhline(0, c="C7")
plt.axvline(4.5, c="C7", ls="--")

plt.legend()
plt.show()

## OLD -- Derivative of healpy maps

In [ ]:
# Plots below optimized for phi=0°, th=45°, sigma=15°
NSIDE = 32
m = gaussian_on_a_sphere(mean_phi=0., mean_th=np.deg2rad(45), NSIDE=NSIDE,
                         sigma=np.deg2rad(15), clip=False)[0]

# Use log priors
m[m <= 0] = np.nextafter(0, 1)
lnm = np.log(m)

m = m

In [ ]:
%%time
alm = hp.map2alm(m)
der = hp.alm2map_der1(alm, NSIDE)

In [ ]:
hp.mollview(der[0])
hp.graticule(verbose=False)
plt.show()

In [ ]:
# Switch sign: d/dth = d/ddec ddec/dth = -d/dec -> dec = np.pi / 2 - th
hp.mollview(-der[1], unit="d/dec")
hp.graticule(verbose=False)
plt.show()

In [ ]:
hp.mollview(der[2], unit="d/ra")
hp.graticule(verbose=False)
plt.show()

In [ ]:
# Compare to missing 1/sin(th) derivative (use small th to see effect)
sin_th = np.sin(hp.pix2ang(NSIDE, np.arange(hp.nside2npix(NSIDE)))[0])
dphi = der[2] * sin_th

hp.mollview(dphi, unit="d/dra no sinTh")
hp.graticule(verbose=False)
plt.show()

In [ ]:
# Compare to a naive finite difference method, which is not really working here
# (with higher NSDIE it gets closer) but shows the same underestimated phi
# derivative as the healpy version
th0 = np.deg2rad(10)
phis = np.linspace(0, 2. * np.pi, 500)

idx = np.unique(hp.ang2pix(NSIDE, th0, phis))
phis = hp.pix2ang(NSIDE, idx)[1]
srt_idx = np.argsort(phis)
phis = phis[srt_idx]
idx = idx[srt_idx]

plt.plot(phis, der[2][idx], label="sinTh")
plt.plot(phis, dphi[idx], label="no sinTh")

num_dphi = np.diff(m[idx]) / np.diff(phis)
mids = 0.5 * (phis[:-1] + phis[1:])
plt.plot(mids, num_dphi, label="num no sinTh", ls="none",
         marker=".", ms=2)
plt.plot(mids, num_dphi / np.sin(th0), label="num sinTh", ls="none",
         marker=".", ms=2)
plt.ylim(-2, 2)
plt.legend()
plt.show()

In [ ]:
# Show interpolation of derivative
th0 = np.deg2rad(50)
phis = np.linspace(0, 2. * np.pi, 500)

idx = hp.ang2pix(hp.get_nside(m), th0, phis)

dphi = der[2]

fig, ax = plt.subplots(1, 1)
ax.plot(phis, dphi[idx], label="mappix")
ax.plot(phis, hp.get_interp_val(m=dphi, phi=phis, theta=th0), label="interp")

# Zoomed insets at min/max regions to show bumpiness of pure map values
# axins = zoomed_inset_axes(ax, zoom=4, loc="upper left", )
axins = inset_axes(ax, width=2, height=1, loc="upper left")
axins.plot(phis, dphi[idx])
axins.plot(phis, hp.get_interp_val(m=dphi, phi=phis, theta=th0))
axins.set_xlim(np.deg2rad(10), np.deg2rad(30))
axins.set_ylim(1.05 * np.amin(dphi[idx]), -4)
plt.xticks(visible=False)
plt.yticks(visible=False)
mark_inset(ax, axins, loc1=3, loc2=4, fc="none", ec="0.5")

axins = inset_axes(ax, width=2, height=1, loc="lower right")
axins.plot(phis, dphi[idx])
axins.plot(phis, hp.get_interp_val(m=dphi, phi=phis, theta=th0))
axins.set_xlim(np.deg2rad(330), np.deg2rad(350))
axins.set_ylim(4, 1.05 * np.amax(dphi[idx]))
plt.xticks(visible=False)
plt.yticks(visible=False)
mark_inset(ax, axins, loc1=1, loc2=2, fc="none", ec="0.5")

ax.legend(bbox_to_anchor=(0.5, 1.15), loc="upper center", ncol=2)
fig.tight_layout()
plt.show()

## OLD -- SoB gradient test plots

In [ ]:
nbg = 10000
nsig = 0
ra0 = np.deg2rad(40)
dec0 = np.deg2rad(0)
ev_ra, ev_dec, ev_sig, mask = make_sample(nbg=nbg, nsig=nsig,
                                          dec0=dec0, ra0=ra0)

In [ ]:
pdf, (sig_grad_ra, sig_grad_dec) = pdf_sig(ra0, dec0, ev_ra, ev_dec, ev_sig)

In [ ]:
pdf, (bg_grad_ra, bg_grad_dec) = pdf_bg(ev_ra, ev_dec)

In [ ]:
sob, (sob_grad_ra, sob_grad_dec) = pdf_sob(ra0, dec0, ev_ra, ev_dec, ev_sig)

In [ ]:
idx = np.argsort(ev_ra)
plt.plot(ev_ra[idx][sob[idx] > 1], sob[idx][sob[idx] > 1], ls="", marker=".")
plt.axvline(ra0, ls="--", c="C7")
plt.xlim(0, 1)
plt.show()

In [ ]:
# Show events in small dec band and scan src ra with fixed events
# The summed gradient should switch at peaks, so that the src ra always wants
# go to the locally densest event cluster.
# Of course we have a lot of local minima, which is why the method doesn't work
# all sky, but only in a very localized region (cut out with a prior for ex.)
idx = np.argsort(ev_ra)
sum_grad = []
ras_ = np.linspace(0., 2. * np.pi, 1000)
for ra0_ in ras_:
    _, (sob_grad_ra, _) = pdf_sob(ra0_, dec0, ev_ra, ev_dec, ev_sig)
    sum_grad.append(np.sum(sob_grad_ra))

# Small band in dec to have the local clustering in ra only
m = (ev_dec > dec0 - np.deg2rad(1)) & (ev_dec < dec0 + np.deg2rad(1))
plt.hist(np.rad2deg(ev_ra[m]), bins=300)
plt.plot(np.rad2deg(ras_), sum_grad / max(sum_grad), "k-")
plt.xlabel("ra")
plt.show()

## OLD -- Signal PDF gradient test plots

In [ ]:
ev_ra = np.deg2rad(65)
ev_dec = np.deg2rad(70)
ev_sig = np.deg2rad(10)

In [ ]:
# Scan in ra
src_ras_ra = np.linspace(-2. * np.pi, 4. * np.pi, 1000)
src_decs_ra = np.ones_like(src_ras_ra) * ev_dec

pdf_ra = pdf_sig(src_ras_ra, src_decs_ra, ev_ra, ev_dec, ev_sig)[0]
plt.plot(src_ras_ra, pdf_ra)
plt.axvline(0, 1, 0, ls="-", c="C7")
plt.axvline(2*np.pi, 1, 0, ls="-", c="C7")
plt.axvline(ev_ra, ls="--", c="C7")
plt.xlabel("ra")
plt.yscale("log", nonposy="mask")
plt.show()

In [ ]:
# Scan in dec
src_decs_dec = np.linspace(-np.pi / 2., np.pi / 2., 1000)
src_ras_dec = np.ones_like(src_decs_dec) * ev_ra

pdf_dec = pdf_sig(src_ras_dec, src_decs_dec, ev_ra, ev_dec, ev_sig)[0]
plt.plot(src_decs_dec, pdf_dec)
plt.axvline(0, 1, 0, ls="-", c="C7")
plt.axvline(ev_dec, ls="--", c="C7")
plt.xlabel("dec")
plt.yscale("log", nonposy="mask")
plt.show()

In [ ]:
# Fit in ra, dec
def fitfun(x, ev_ra, ev_dec, ev_sig):
    pdf, grad = pdf_sig(x[0], x[1], ev_ra, ev_dec, ev_sig)
    return -pdf, -grad
    
x0 = np.deg2rad([60, 60])
res = sco.minimize(fitfun, x0=x0, bounds=[[-2*np.pi, 4*np.pi],
                                          [-np.pi/2., np.pi/2.]],
             jac=True, options={"ftol": 1e-12, "gtol": 1e-12}, tol=1e-12,
             args=(ev_ra, ev_dec, ev_sig))

ra_bf = wrap_angle(res.x[0], wrap_angle=2*np.pi)
dec_bf = res.x[1]

In [ ]:
print(res)
print(u"\nBest fit: RA = {:.2f}°, DEC = {:.2f}°".format(*np.rad2deg([ra_bf,
                                                                    dec_bf])))
print(u"\nFit seed: RA = {:.2f}°, DEC = {:.2f}°".format(*np.rad2deg(x0)))
print(u"\nTruth   : RA = {:.2f}°, DEC = {:.2f}°".format(*np.rad2deg([ev_ra,
                                                                    ev_dec])))

In [ ]:
# Scan ra gradient
d_pdf = np.diff(pdf_ra) / np.diff(src_ras_ra)

grad_ra = pdf_sig(src_ras_ra, src_decs_ra, ev_ra, ev_dec, ev_sig)[1][0]
mids = 0.5 * (src_ras_ra[:-1] + src_ras_ra[1:])

plt.plot(src_ras_ra, grad_ra)
plt.plot(mids, d_pdf, ls="--")
plt.axhline(0, 1, 0, ls="-", c="C7")
plt.axvline(0, 1, 0, ls="-", c="C7")
plt.axvline(2*np.pi, 1, 0, ls="-", c="C7")

plt.axvline(res.x[0], 0, 1, ls="-.", c="C2")
plt.axvline(ra_bf, 0, 1, ls="-.", c="C2")
plt.axvline(ev_ra, 1, 0, ls="--", c="C7")

plt.title(u"Best fit RA = {:.2f}°".format(np.rad2deg(ra_bf)))
plt.xlabel("ra")
plt.yscale("symlog", linthreshy=1e-2)
plt.show()

In [ ]:
# Scan dec gradient
d_pdf = np.diff(pdf_dec) / np.diff(src_decs_dec)

grad_dec = pdf_sig(src_ras_dec, src_decs_dec, ev_ra, ev_dec, ev_sig)[1][1]
mids = 0.5 * (src_decs_dec[:-1] + src_decs_dec[1:])

plt.plot(src_decs_dec, grad_dec)
plt.plot(mids, d_pdf, ls="--")
plt.axhline(0, 1, 0, ls="-", c="C7")
plt.axvline(-np.pi / 2., 1, 0, ls="-", c="C7")
plt.axvline(np.pi / 2., 1, 0, ls="-", c="C7")

plt.axvline(res.x[1], 0, 1, ls="-.", c="C2")
plt.axvline(dec_bf, 0, 1, ls="-.", c="C2")
plt.axvline(ev_dec, 1, 0, ls="--", c="C7")

plt.title(u"Best fit DEC = {:.2f}°".format(np.rad2deg(dec_bf)))
plt.xlabel("dec")
plt.yscale("symlog", linthreshy=1e-2)
plt.show()

## OLD -- Zoom maps

In [ ]:
ra0, dec0 = [0.5 * np.pi, 0.5 * np.pi / 2.]
th0, phi0 = DecRaToThetaPhi(dec0, ra0)
m = gaussian_on_a_sphere(clip=False, NSIDE=128, sigma=np.deg2rad(5),
                         mean_phi=phi0, mean_th=th0)[0]

fig, ax = plt.subplots(1, 1)
radius = np.deg2rad(20)
# cartzoom_radius(m, r=radius, center=[ra0, dec0])
# cartzoom(m, dec_rng=[np.deg2rad(40), np.deg2rad(50)],
#          ra_rng=[np.deg2rad(95), np.deg2rad(105)])
cartzoom(m)
make_astro_xaxis(ax, time_xax=True)
ax.grid(which="both")
plt.show()

lon, lat = None, None
hp.cartview(m, cmap=plt.cm.viridis)  # , lonra=[-90, 0], latra=[0, 90])
hp.graticule(color="w", verbose=False)
plt.show()

## OLD -- Prior map tests

In [ ]:
phi0, th0 = np.pi / 2., np.pi / 2.
m = gaussian_on_a_sphere(mean_th=th0, mean_phi=phi0, clip=False,
                         NSIDE=32, sigma=np.deg2rad(5))[0]
m[m <= 0] = np.nextafter(0, 1)  # Set 0 to minimal machine val > 0 for log

In [ ]:
# Scan theta with fixed phi0
ths = np.linspace(0, np.pi, 1000)
vals = hp.get_interp_val(m=np.log(m), phi=phi0, theta=ths)

plt.plot(ths, np.log(m[hp.ang2pix(nside=hp.get_nside(m),
                                  theta=ths, phi=phi0)]))
plt.plot(ths, vals, label="interp")
plt.xlabel("theta")
plt.ylabel("lnPDF")
plt.legend()
plt.show()

In [ ]:
# Scan phi in multiple thetas
phis = np.linspace(0, 2. * np.pi, 1000)
th0s = np.linspace(0, np.pi / 2., 20)
cols = plt.cm.viridis(th0s / np.amax(th0s))
for th0, c in zip(th0s, cols):
    plt.plot(phis, np.log(m[hp.ang2pix(nside=hp.get_nside(m),
                                       theta=th0, phi=phis)]), c=c)
    plt.plot(phis, np.log(hp.get_interp_val(m=m, phi=phis, theta=th0)),
             c="k", label=r"${:.1f}\pi$".format(th0 / np.pi), ls="-", lw=1)

# plt.legend(bbox_to_anchor=(1, 1), loc="upper left", ncol=2)
plt.show()

In [ ]:
cmap = plt.cm.viridis
cmap.set_under("w")
hp.mollview(np.log(m), cmap=cmap, unit="lnPDF")
for th0 in th0s:
    hp.projplot(np.repeat([th0], 100), np.linspace(0, 2 * np.pi, 100),
                "w--", ms=1)
hp.graticule(verbose=False, color="w")
hp.projplot(th0, phi0, "k*")
plt.show()

Linear test

In [ ]:
m = np.arange(12 * 8**2)
phis = np.linspace(0, 2. * np.pi, 1000)
th0 = np.pi / 2.
vals = hp.get_interp_val(m=m, phi=phis, theta=th0)

plt.plot(phis, vals)
plt.plot(phis, m[hp.ang2pix(nside=hp.get_nside(m), theta=th0, phi=phis)])
plt.show()

In [ ]:
ths = np.linspace(0, np.pi, 1000)
phi0 = 1.7 * np.pi
vals = hp.get_interp_val(m=m, phi=phi0, theta=ths)

plt.plot(ths, vals)
plt.plot(ths, m[hp.ang2pix(nside=hp.get_nside(m), theta=ths, phi=np.pi)])
plt.show()

# Ana

## LLH

In [ ]:
%%time
# Just assume a source uncertainty here. Signal injection is not adapted yet
NSIDE = 256
ra0 = np.deg2rad([120, 150, 300])
dec0 = np.deg2rad([-10, 20, 25])  # True src positions
th0, phi0 = DecRaToThetaPhi(dec0, ra0)
sigma0 = np.atleast_1d(np.deg2rad(2))
if len(sigma0) != len(ra0) and len(sigma0) == 1:
    sigma0 = np.repeat([sigma0], repeats=len(ra0))
    
ln_priors = []
ln_priors_dphi = []
ln_priors_dth = []
for i, (th0i, phi0i, sigma0i) in enumerate(zip(th0, phi0, sigma0)):
    print(u"Making prior at: PHI={:.2f}, TH={:.2f}".format(phi0i, th0i))
    prior = gaussian_on_a_sphere(mean_th=th0i, mean_phi=phi0i, 
                                 clip=5., NSIDE=NSIDE, sigma=sigma0i)[0]
    # Avoid taking the log of 0, use the smallest valid map number instead
    prior[prior <= 0] = np.amin(prior[prior > 0])
    ln_prior = np.log(prior)

    # Make ra0, dec0 more realistic by setting it to the map maximum as in HESE
    idx = np.argmax(ln_prior)
    th0[i], phi0[i] = hp.pix2ang(NSIDE, idx)
    dec0[i], ra0[i] = ThetaPhiToDecRa(th0[i], phi0[i])

    # Get the log derivatives from the finite elements method
    ln_prior_dphi, ln_prior_dth = map_der(ln_prior)
    
    ln_priors.append(ln_prior)
    ln_priors_dphi.append(ln_prior_dphi)
    ln_priors_dth.append(ln_prior_dth)

In [ ]:
# Create a test sample
rnd_gen = np.random.RandomState(137462)
nbg = int(1e5)
nsig = np.ones_like(ra0) * 10
sig = {"nsig": nsig, "ra": ra0, "dec": dec0, "sigma": sigma0}

ras, decs, sigmas, sig_idx = make_sample(nbg=nbg, sig=sig, rnd=rnd_gen)

data = (ras, decs, sigmas)
lnpriormap = (np.sum(ln_priors_dphi, axis=0), ln_priors_dth, ln_priors_dphi)

First try to fit the source position with a prior map

In [ ]:
res = fit_llh_ratio(x0=[10., np.deg2rad(302), np.deg2rad(-6)], fix_ra_dec=False,
                    data=data, lnpriormap=lnpriormap)

In [ ]:
res

In [ ]:
ns = np.linspace(0, 5, 100)
llh = np.zeros_like(ns)
grad = np.zeros((len(ns), 3), dtype=float)
for i in range(len(ns)):
#     llh[i], grad[i] = llh_ratio((ns[i], np.deg2rad(302.5), np.deg2rad(-9)), data, lnpriormap)
    llh[i], grad[i] = llh_ratio((ns[i], res.x[1], res.x[2]), data, lnpriormap)

In [ ]:
plt.plot(ns, -llh)
plt.plot(ns, -grad[:, 0])

plt.axhline(0, c="C7")
plt.axvline(res.x[0], c="k")
plt.axvline(ns[np.argmax(llh)], ls="--", c="C3")
plt.show()

Scan to get an overview of the fit region

In [ ]:
%%time
# Scan the LLH in ras, decs for a given event combination
src_idx = 2
NSIDE = 64
radius = 4. * sigma0[src_idx]

# ns_fix = [res.x[0]]
# ns_fix = np.linspace(0, 10, 11)
# ns_fix = [False]
ns_fix = [1]
nss = []
llhs = []
grads = []
for nsfi in tqdm(ns_fix):
    nsi, llhi, gradi = sky_scan(NSIDE, data=data,
                                lnpriormap=lnpriormap[src_idx],
                                ra0=ra0[src_idx], dec0=dec0[src_idx],
                                radius=radius, ns_fix=nsfi)
    nss.append(nsi)
    llhs.append(llhi)
    grads.append(gradi)

## Plots

In [ ]:
for llh in llhs:
    fig, ax, img = cartzoom_radius(llh, center=[ra0[src_idx], dec0[src_idx]],
                                   r=radius, cmap=ps_cmap)
    make_astro_xaxis(ax, time_xax=False)
    ax.grid()
    ax.plot(ra0[src_idx], dec0[src_idx], "ko", mfc="none", ms=10)  # Injection
    
    # Scan best pix
    max_idx = np.argmax(llh)
    dec_max, ra_max = ThetaPhiToDecRa(*hp.pix2ang(NSIDE, max_idx))
    ax.plot(ra_max, dec_max, "k+", mfc="none", ms=9)  # Scan best fit
    
    # Fit best position
#     ax.plot(res.x[1], res.x[2], "k*", ms=10)
    
    ax.set_title(u"Radius {:.2f}°".format(np.rad2deg(radius)))
    # plt.savefig("single_src_w_injection.png", dpi=150)
    fig.colorbar(img, ax=ax)
    plt.show()

In [ ]:
for i, grad in enumerate(grads):
    # Maps for TS, ns mutiplied with prior in normal space
    fig, ax, img = cartzoom_radius(grad[:, 0],
                                   center=[ra0[src_idx], dec0[src_idx]],
                                   r=radius, cmap="seismic")
    make_astro_xaxis(ax, time_xax=False)
    ax.grid()
    ax.plot(ra0[src_idx], dec0[src_idx], "ko", mfc="none", ms=10)  # Injection
    
    # Scan best pix
    max_idx = np.argmax(llhs[i])
    dec_max, ra_max = ThetaPhiToDecRa(*hp.pix2ang(NSIDE, max_idx))
    ax.plot(ra_max, dec_max, "k+", mfc="none", ms=9)  # Scan best fit
    
    # Fit best position
#     ax.plot(res.x[1], res.x[2], "k*", ms=10)
    
    ax.set_title(u"Radius {:.2f}°".format(np.rad2deg(radius)))
    # plt.savefig("single_src_w_injection.png", dpi=150)
    fig.colorbar(img, ax=ax)
    plt.show()

In [ ]:
# Maps for TS, ns mutiplied with prior in normal space
fig, ax, img = cartzoom_radius(llh, center=[ra0, dec0], r=radius, cmap=ps_cmap)
make_astro_xaxis(ax, time_xax=False)
ax.grid()
ax.plot(ra0, dec0, "ko", mfc="none", ms=10)  # Injection
ax.set_title(u"Radius {:.2f}°".format(np.rad2deg(radius)))
# plt.savefig("single_src_w_injection.png", dpi=150)
fig.colorbar(img, ax=ax)
plt.show()

In [ ]:
# Maps for TS, ns mutiplied with prior in normal space
fig, ax, img = cartzoom_radius(llh, center=[ra0, dec0], r=radius, cmap=ps_cmap)
make_astro_xaxis(ax, time_xax=False)
ax.grid()
ax.plot(ra0, dec0, "ko", mfc="none", ms=10)  # Injection
ax.set_title(u"Radius {:.2f}°".format(np.rad2deg(radius)))
# plt.savefig("single_src_w_injection.png", dpi=150)
fig.colorbar(img, ax=ax)
plt.show()

In [ ]:
# Maps for TS, ns mutiplied with prior in normal space
fig, ax, img = cartzoom_radius(np.log(prior), center=[ra0, dec0], r=radius,
                               cmap=plt.cm.Greens_r)
make_astro_xaxis(ax, time_xax=False)
plt.grid()
plt.plot(ra0_, dec0_, "ko", mfc="none")  # Injection
plt.plot(res.x[1], res.x[2], "k*", mfc="none", ms=10)  # Best fit
plt.plot(ra_scan, dec_scan, "kd", mfc="none", ms=10)  # Best fit from scan
plt.title(u"Radius {:.2f}°".format(np.rad2deg(radius)))
# plt.savefig("single_src_w_injection.png", dpi=150)
plt.colorbar(img, ax=ax)
plt.show()

In [ ]:
# Maps for TS, ns mutiplied with prior in normal space
fig, ax, img = cartzoom_radius(llh, center=[ra0, dec0], r=radius,
                               cmap=plt.cm.Greens_r)
make_astro_xaxis(ax, time_xax=False)
plt.grid()
plt.plot(ra0_, dec0_, "ko", mfc="none")  # Injection
plt.plot(res.x[1], res.x[2], "k*", mfc="none", ms=10)  # Best fit
plt.plot(ra_scan, dec_scan, "kd", mfc="none", ms=10)  # Best fit from scan
plt.title(u"Radius {:.2f}°".format(np.rad2deg(radius)))
# plt.savefig("single_src_w_injection.png", dpi=150)
plt.colorbar(img, ax=ax)
plt.show()

In [ ]:
# Maps for TS, ns mutiplied with prior in normal space
llh_cut = np.zeros_like(llh)
_conf = 2**2 / 2  # Wilk's confidence: r sigma = r^2/2 
llh_cut[llh>np.amax(llh) - _conf] = llh[llh>np.amax(llh) - _conf]
fig, ax, img = cartzoom_radius(llh_cut, center=[ra0, dec0], r=radius,
                               cmap=plt.cm.Greens_r)
make_astro_xaxis(ax, time_xax=False)
plt.grid()
plt.plot(ra0_, dec0_, "ko", mfc="none")  # Injection
plt.plot(res.x[1], res.x[2], "k*", mfc="none", ms=10)  # Best fit
plt.plot(ra_scan, dec_scan, "kd", mfc="none", ms=10)  # Best fit from scan
plt.title(u"Radius {:.2f}°. $3\sigma$ Wilk's contours".format(
    np.rad2deg(radius)))
# plt.savefig("single_src_w_injection.png", dpi=150)
plt.colorbar(img, ax=ax)
plt.show()

In [ ]:
# Plot event view
th_bg, phi_bg = DecRaToThetaPhi(decs[~sig_idx], ras[~sig_idx])
th_sig, phi_sig = DecRaToThetaPhi(decs[sig_idx], ras[sig_idx])

hp.graticule(verbose=False)
hp.projscatter(th_bg, phi_bg, s=sigmas[~sig_idx] * 100, c="C7")
hp.projscatter(th_sig, phi_sig, s=sigmas[sig_idx] * 100, c="C3")
hp.projscatter(th0, phi0, marker="o", s=250, color="k", facecolor="", lw=1.5)
plt.show()

In [ ]:
# Maps for TS, ns mutiplied with prior in normal space
hp.mollview(llh, cmap=ps_cmap, min=0, unit="TS")
hp.graticule(verbose=False)
hp.projscatter(th0, phi0, marker="o", s=200, color="k", facecolor="")
plt.title("mollweide equatorial")
plt.show()

hp.mollview(ns, cmap=ps_cmap, min=0, unit="ns")
hp.graticule(verbose=False)
hp.projscatter(th0, phi0, marker="o", s=200, color="k", facecolor="")
plt.title("mollweide equatorial")
plt.show()